Complete Time Series Event Labeling System
==========================================

Input:  [B, L] time series tensor
Output: Rich event annotations at multiple scales

Components:
1. Multi-scale feature extraction
2. Step-wise symbolic labels
3. Trend segment detection  
4. Peak/trough detection
5. Volatility regime detection
6. Change point detection
7. Motif discovery
8. Hierarchical event fusion


In [20]:
import torch
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
from scipy import signal as scipy_signal
from scipy.stats import zscore
import warnings
warnings.filterwarnings('ignore')

In [21]:
# ============================================================================
# 1. COMPREHENSIVE VOCABULARY (110+ labels)
# ============================================================================

@dataclass
class EventVocabulary:
    """Complete event vocabulary with hierarchical structure"""
    
    # Special tokens
    PAD = 0
    MASK = 1
    UNK = 2
    
    # Step-level movement (3-12)
    FLAT = 3
    UP_TINY = 4
    UP_SMALL = 5
    UP_MEDIUM = 6
    UP_LARGE = 7
    UP_HUGE = 8
    DOWN_TINY = 9
    DOWN_SMALL = 10
    DOWN_MEDIUM = 11
    DOWN_LARGE = 12
    DOWN_HUGE = 13
    
    # Spikes (14-19)
    SPIKE_UP_WEAK = 14
    SPIKE_UP_STRONG = 15
    SPIKE_UP_EXTREME = 16
    SPIKE_DOWN_WEAK = 17
    SPIKE_DOWN_STRONG = 18
    SPIKE_DOWN_EXTREME = 19
    
    # Trend segments (20-39)
    UPTREND_SHORT_WEAK = 20
    UPTREND_SHORT_MODERATE = 21
    UPTREND_SHORT_STRONG = 22
    UPTREND_MEDIUM_WEAK = 23
    UPTREND_MEDIUM_MODERATE = 24
    UPTREND_MEDIUM_STRONG = 25
    UPTREND_LONG_WEAK = 26
    UPTREND_LONG_MODERATE = 27
    UPTREND_LONG_STRONG = 28
    ACCELERATING_UP = 29
    DECELERATING_UP = 30
    
    DOWNTREND_SHORT_WEAK = 31
    DOWNTREND_SHORT_MODERATE = 32
    DOWNTREND_SHORT_STRONG = 33
    DOWNTREND_MEDIUM_WEAK = 34
    DOWNTREND_MEDIUM_MODERATE = 35
    DOWNTREND_MEDIUM_STRONG = 36
    DOWNTREND_LONG_WEAK = 37
    DOWNTREND_LONG_MODERATE = 38
    DOWNTREND_LONG_STRONG = 39
    ACCELERATING_DOWN = 40
    DECELERATING_DOWN = 41
    
    # Flat/stable (42-46)
    FLAT_SHORT = 42
    FLAT_MEDIUM = 43
    FLAT_LONG = 44
    PLATEAU = 45
    STABLE_REGIME = 46
    
    # Peaks and troughs (47-62)
    LOCAL_PEAK_WEAK = 47
    LOCAL_PEAK_MODERATE = 48
    LOCAL_PEAK_STRONG = 49
    SHARP_PEAK = 50
    BROAD_PEAK = 51
    DOUBLE_TOP = 52
    TRIPLE_TOP = 53
    ROUND_TOP = 54
    
    LOCAL_TROUGH_WEAK = 55
    LOCAL_TROUGH_MODERATE = 56
    LOCAL_TROUGH_STRONG = 57
    SHARP_TROUGH = 58
    BROAD_TROUGH = 59
    DOUBLE_BOTTOM = 60
    TRIPLE_BOTTOM = 61
    ROUND_BOTTOM = 62
    
    # Volatility regimes (63-72)
    LOW_VOLATILITY = 63
    NORMAL_VOLATILITY = 64
    ELEVATED_VOLATILITY = 65
    HIGH_VOLATILITY = 66
    VOLATILITY_SPIKE = 67
    VOLATILITY_CLUSTER = 68
    CALM_PERIOD = 69
    TURBULENT_PERIOD = 70
    VOLATILITY_TRANSITION_UP = 71
    VOLATILITY_TRANSITION_DOWN = 72
    
    # Oscillations (73-80)
    OSCILLATION_REGULAR_SMALL = 73
    OSCILLATION_REGULAR_MEDIUM = 74
    OSCILLATION_REGULAR_LARGE = 75
    OSCILLATION_IRREGULAR = 76
    HIGH_FREQUENCY_NOISE = 77
    CYCLIC_PATTERN = 78
    DAMPENED_OSCILLATION = 79
    AMPLIFYING_OSCILLATION = 80
    
    # Change points & regime shifts (81-90)
    MEAN_SHIFT_UP = 81
    MEAN_SHIFT_DOWN = 82
    VARIANCE_INCREASE = 83
    VARIANCE_DECREASE = 84
    LEVEL_SHIFT_UP = 85
    LEVEL_SHIFT_DOWN = 86
    STRUCTURAL_BREAK = 87
    REGIME_CHANGE = 88
    TREND_REVERSAL = 89
    SUDDEN_CHANGE = 90
    
    # Structural patterns (91-105)
    RALLY_THEN_DROP = 91
    DROP_THEN_RALLY = 92
    UP_THEN_SIDEWAYS = 93
    DOWN_THEN_SIDEWAYS = 94
    STAIRCASE_UP = 95
    STAIRCASE_DOWN = 96
    V_SHAPE_RECOVERY = 97
    INVERTED_V_SHAPE = 98
    W_PATTERN = 99
    M_PATTERN = 100
    CUP_AND_HANDLE = 101
    HEAD_AND_SHOULDERS = 102
    ROUNDING_BOTTOM = 103
    ROUNDING_TOP = 104
    CONSOLIDATION = 105
    
    # Anomalies (106-112)
    OUTLIER_HIGH = 106
    OUTLIER_LOW = 107
    RARE_EVENT = 108
    DISCORD = 109
    ANOMALOUS_SEGMENT = 110
    UNEXPECTED_BEHAVIOR = 111
    DATA_QUALITY_ISSUE = 112
    
    # Macro regimes (113-120)
    BULLISH_REGIME = 113
    BEARISH_REGIME = 114
    SIDEWAYS_REGIME = 115
    VOLATILE_REGIME = 116
    TRENDING_REGIME = 117
    MEAN_REVERTING_REGIME = 118
    MOMENTUM_REGIME = 119
    TRANSITION_REGIME = 120
    
    @classmethod
    def get_vocab_size(cls):
        return 121
    
    @classmethod
    def id_to_label(cls, idx: int) -> str:
        for name, value in vars(cls).items():
            if isinstance(value, int) and value == idx:
                return name
        return "UNKNOWN"

VOCAB = EventVocabulary()

In [22]:
# ============================================================================
# 2. MULTI-SCALE FEATURE EXTRACTOR (FIXED)
# ============================================================================

class MultiScaleFeatureExtractor:
    """Extract features at multiple temporal scales"""
    
    def __init__(self, scales=[5, 10, 20, 50, 100]):
        self.scales = scales
    
    def extract_features(self, x: torch.Tensor) -> Dict[str, torch.Tensor]:
        """
        x: [B, L] time series
        Returns: dict of features at multiple scales
        """
        B, L = x.shape
        device = x.device
        
        features = {}
        
        # First and second derivatives
        dx = torch.diff(x, dim=1)  # [B, L-1]
        features['dx'] = F.pad(dx, (1, 0), value=0)  # [B, L]
        
        d2x = torch.diff(dx, dim=1)  # [B, L-2]
        features['d2x'] = F.pad(d2x, (2, 0), value=0)  # [B, L]
        
        # Multi-scale rolling features
        for w in self.scales:
            if w >= L:
                continue
            
            # Prepare for conv1d: [B, 1, L]
            x_3d = x.unsqueeze(1)  # [B, 1, L]
            
            # Rolling mean using conv1d
            kernel = torch.ones(1, 1, w, device=device) / w  # [1, 1, w]
            
            # Pad to maintain length
            padding = w - 1
            x_padded = F.pad(x_3d, (padding, 0), mode='replicate')  # [B, 1, L+w-1]
            
            rolling_mean_3d = F.conv1d(x_padded, kernel)  # [B, 1, L]
            rolling_mean = rolling_mean_3d.squeeze(1)  # [B, L]
            features[f'mean_{w}'] = rolling_mean
            
            # Rolling std (volatility)
            # Compute (x - mean)^2 for each point
            x_centered = x_3d - rolling_mean.unsqueeze(1)  # [B, 1, L]
            x_centered_padded = F.pad(x_centered, (padding, 0), mode='replicate')  # [B, 1, L+w-1]
            
            rolling_var_3d = F.conv1d(x_centered_padded ** 2, kernel)  # [B, 1, L]
            rolling_var = rolling_var_3d.squeeze(1)  # [B, L]
            features[f'std_{w}'] = torch.sqrt(rolling_var.clamp(min=1e-8))
            
            # Rolling slope (linear regression in window)
            slopes = self._rolling_slope(x, w)
            features[f'slope_{w}'] = slopes
            
            # Energy (sum of absolute changes)
            dx_abs = torch.abs(features['dx']).unsqueeze(1)  # [B, 1, L]
            dx_abs_padded = F.pad(dx_abs, (padding, 0), mode='replicate')  # [B, 1, L+w-1]
            energy_3d = F.conv1d(dx_abs_padded, torch.ones(1, 1, w, device=device))  # [B, 1, L]
            features[f'energy_{w}'] = energy_3d.squeeze(1)  # [B, L]
        
        # Z-scores for anomaly detection
        mean = x.mean(dim=1, keepdim=True)
        std = x.std(dim=1, keepdim=True).clamp(min=1e-8)
        features['zscore'] = (x - mean) / std
        
        return features
    
    def _rolling_slope(self, x: torch.Tensor, window: int) -> torch.Tensor:
        """Compute rolling linear regression slope"""
        B, L = x.shape
        device = x.device
        
        slopes = torch.zeros(B, L, device=device)
        
        # Create time indices for regression
        t = torch.arange(window, dtype=x.dtype, device=device)
        t_mean = t.mean()
        t_var = ((t - t_mean) ** 2).sum()
        
        # Use rolling window
        for i in range(L):
            start = max(0, i - window + 1)
            end = i + 1
            actual_window = end - start
            
            if actual_window < 3:  # Need minimum points for regression
                continue
            
            # Get window data
            window_data = x[:, start:end]  # [B, actual_window]
            
            # Time indices for this window
            t_win = torch.arange(actual_window, dtype=x.dtype, device=device)
            t_win_mean = t_win.mean()
            t_win_var = ((t_win - t_win_mean) ** 2).sum()
            
            # Compute slope
            x_mean = window_data.mean(dim=1, keepdim=True)
            cov = ((t_win - t_win_mean) * (window_data - x_mean)).sum(dim=1)
            slopes[:, i] = cov / (t_win_var + 1e-8)
        
        return slopes

In [23]:
# ============================================================================
# 3. STEP-WISE SYMBOLIC ENCODER
# ============================================================================

class StepWiseEncoder:
    """Encode each timestep with symbolic labels"""
    
    def encode(self, x: torch.Tensor, features: Dict) -> torch.Tensor:
        """
        x: [B, L]
        features: dict from MultiScaleFeatureExtractor
        Returns: [B, L] step labels
        """
        B, L = x.shape
        device = x.device
        
        # Compute global quantiles for thresholding
        dx = features['dx']
        abs_dx = torch.abs(dx[:, 1:])  # Skip first padding
        
        if abs_dx.numel() == 0:
            return torch.full((B, L), VOCAB.FLAT, dtype=torch.long, device=device)
        
        # Quantiles: 20%, 40%, 60%, 80%, 95%
        quantiles = torch.quantile(
            abs_dx.reshape(-1),
            torch.tensor([0.20, 0.40, 0.60, 0.80, 0.95], device=device)
        )
        q20, q40, q60, q80, q95 = quantiles
        
        epsilon = 0.1 * q20  # Flat threshold
        
        # Initialize with PAD
        labels = torch.full((B, L), VOCAB.PAD, dtype=torch.long, device=device)
        
        # Process t >= 1
        for t in range(1, L):
            diff = dx[:, t]
            abs_diff = torch.abs(diff)
            
            # Flat
            flat_mask = abs_diff < epsilon
            labels[flat_mask, t] = VOCAB.FLAT
            
            # Upward movements
            up_mask = (diff > 0) & (~flat_mask)
            labels[up_mask & (abs_diff <= q20), t] = VOCAB.UP_TINY
            labels[up_mask & (abs_diff > q20) & (abs_diff <= q40), t] = VOCAB.UP_SMALL
            labels[up_mask & (abs_diff > q40) & (abs_diff <= q60), t] = VOCAB.UP_MEDIUM
            labels[up_mask & (abs_diff > q60) & (abs_diff <= q80), t] = VOCAB.UP_LARGE
            labels[up_mask & (abs_diff > q80) & (abs_diff <= q95), t] = VOCAB.UP_HUGE
            
            # Extreme spikes
            labels[up_mask & (abs_diff > q95), t] = VOCAB.SPIKE_UP_EXTREME
            
            # Downward movements
            down_mask = (diff < 0) & (~flat_mask)
            labels[down_mask & (abs_diff <= q20), t] = VOCAB.DOWN_TINY
            labels[down_mask & (abs_diff > q20) & (abs_diff <= q40), t] = VOCAB.DOWN_SMALL
            labels[down_mask & (abs_diff > q40) & (abs_diff <= q60), t] = VOCAB.DOWN_MEDIUM
            labels[down_mask & (abs_diff > q60) & (abs_diff <= q80), t] = VOCAB.DOWN_LARGE
            labels[down_mask & (abs_diff > q80) & (abs_diff <= q95), t] = VOCAB.DOWN_HUGE
            labels[down_mask & (abs_diff > q95), t] = VOCAB.SPIKE_DOWN_EXTREME
        
        return labels

In [24]:
# ============================================================================
# 4. TREND SEGMENT DETECTOR
# ============================================================================

@dataclass
class TrendSegment:
    start: int
    end: int
    direction: str  # 'up', 'down', 'flat'
    slope: float
    strength: str  # 'weak', 'moderate', 'strong'
    label: int

class TrendSegmentDetector:
    """Detect trend segments using piecewise linear approximation"""
    
    def detect(self, x: torch.Tensor, features: Dict, 
               sample_idx: int = 0) -> List[TrendSegment]:
        """
        Detect trends for a single sequence
        x: [L] single sequence
        """
        x_np = x.cpu().numpy()
        L = len(x_np)
        
        segments = []
        
        # Use rolling slope features
        if 'slope_20' in features:
            slopes = features['slope_20'][sample_idx].cpu().numpy()
        else:
            # Fallback: compute simple slopes
            slopes = np.gradient(x_np)
        
        # Find runs of same sign
        slope_sign = np.sign(slopes)
        slope_sign[np.abs(slopes) < 0.01] = 0  # Flat threshold
        
        # Detect sign changes
        changes = np.where(np.diff(slope_sign) != 0)[0] + 1
        breakpoints = np.concatenate([[0], changes, [L]])
        
        for i in range(len(breakpoints) - 1):
            start = breakpoints[i]
            end = breakpoints[i + 1]
            
            if end - start < 3:  # Skip very short segments
                continue
            
            # Segment statistics
            seg_slope = slopes[start:end].mean()
            seg_std = slopes[start:end].std()
            duration = end - start
            
            # Classify direction
            if abs(seg_slope) < 0.01:
                direction = 'flat'
                label = self._classify_flat(duration)
            elif seg_slope > 0:
                direction = 'up'
                strength, accel = self._classify_strength(seg_slope, slopes[start:end])
                label = self._classify_uptrend(duration, strength, accel)
            else:
                direction = 'down'
                strength, accel = self._classify_strength(abs(seg_slope), slopes[start:end])
                label = self._classify_downtrend(duration, strength, accel)
            
            segments.append(TrendSegment(
                start=start,
                end=end,
                direction=direction,
                slope=seg_slope,
                strength=strength if direction != 'flat' else 'n/a',
                label=label
            ))
        
        return segments
    
    def _classify_strength(self, avg_slope: float, slopes: np.ndarray):
        """Classify trend strength and acceleration"""
        if avg_slope < 0.05:
            strength = 'weak'
        elif avg_slope < 0.15:
            strength = 'moderate'
        else:
            strength = 'strong'
        
        # Check acceleration
        mid = len(slopes) // 2
        slope_first_half = slopes[:mid].mean()
        slope_second_half = slopes[mid:].mean()
        
        if abs(slope_second_half) > abs(slope_first_half) * 1.2:
            accel = 'accelerating'
        elif abs(slope_second_half) < abs(slope_first_half) * 0.8:
            accel = 'decelerating'
        else:
            accel = 'steady'
        
        return strength, accel
    
    def _classify_uptrend(self, duration: int, strength: str, accel: str) -> int:
        """Get label ID for uptrend"""
        if accel == 'accelerating':
            return VOCAB.ACCELERATING_UP
        elif accel == 'decelerating':
            return VOCAB.DECELERATING_UP
        
        # Duration classification
        if duration < 20:
            dur_type = 'SHORT'
        elif duration < 50:
            dur_type = 'MEDIUM'
        else:
            dur_type = 'LONG'
        
        # Strength
        str_type = strength.upper()
        
        # Map to vocabulary
        label_name = f"UPTREND_{dur_type}_{str_type}"
        return getattr(VOCAB, label_name, VOCAB.UPTREND_MEDIUM_MODERATE)
    
    def _classify_downtrend(self, duration: int, strength: str, accel: str) -> int:
        """Get label ID for downtrend"""
        if accel == 'accelerating':
            return VOCAB.ACCELERATING_DOWN
        elif accel == 'decelerating':
            return VOCAB.DECELERATING_DOWN
        
        if duration < 20:
            dur_type = 'SHORT'
        elif duration < 50:
            dur_type = 'MEDIUM'
        else:
            dur_type = 'LONG'
        
        str_type = strength.upper()
        label_name = f"DOWNTREND_{dur_type}_{str_type}"
        return getattr(VOCAB, label_name, VOCAB.DOWNTREND_MEDIUM_MODERATE)
    
    def _classify_flat(self, duration: int) -> int:
        """Get label for flat segment"""
        if duration < 15:
            return VOCAB.FLAT_SHORT
        elif duration < 40:
            return VOCAB.FLAT_MEDIUM
        else:
            return VOCAB.FLAT_LONG

In [25]:
# ============================================================================
# 5. PEAK/TROUGH DETECTOR
# ============================================================================

@dataclass
class PeakTroughEvent:
    index: int
    type: str  # 'peak' or 'trough'
    prominence: float
    label: int

class PeakTroughDetector:
    """Detect peaks and troughs using scipy"""
    
    def detect(self, x: torch.Tensor, sample_idx: int = 0) -> List[PeakTroughEvent]:
        """Detect peaks/troughs in single sequence"""
        x_np = x.cpu().numpy()
        events = []
        
        # Detect peaks
        peaks, properties = scipy_signal.find_peaks(
            x_np, 
            prominence=0.1,
            width=2
        )
        
        for idx, prom in zip(peaks, properties['prominences']):
            label = self._classify_peak(prom, properties['widths'][list(peaks).index(idx)])
            events.append(PeakTroughEvent(
                index=int(idx),
                type='peak',
                prominence=float(prom),
                label=label
            ))
        
        # Detect troughs (peaks of inverted signal)
        troughs, properties = scipy_signal.find_peaks(
            -x_np,
            prominence=0.1,
            width=2
        )
        
        for idx, prom in zip(troughs, properties['prominences']):
            label = self._classify_trough(prom, properties['widths'][list(troughs).index(idx)])
            events.append(PeakTroughEvent(
                index=int(idx),
                type='trough',
                prominence=float(prom),
                label=label
            ))
        
        # Sort by index
        events.sort(key=lambda e: e.index)
        
        return events
    
    def _classify_peak(self, prominence: float, width: float) -> int:
        """Classify peak type"""
        if prominence < 0.5:
            return VOCAB.LOCAL_PEAK_WEAK
        elif prominence < 1.5:
            if width < 5:
                return VOCAB.SHARP_PEAK
            elif width > 15:
                return VOCAB.BROAD_PEAK
            else:
                return VOCAB.LOCAL_PEAK_MODERATE
        else:
            if width > 20:
                return VOCAB.ROUND_TOP
            else:
                return VOCAB.LOCAL_PEAK_STRONG
    
    def _classify_trough(self, prominence: float, width: float) -> int:
        """Classify trough type"""
        if prominence < 0.5:
            return VOCAB.LOCAL_TROUGH_WEAK
        elif prominence < 1.5:
            if width < 5:
                return VOCAB.SHARP_TROUGH
            elif width > 15:
                return VOCAB.BROAD_TROUGH
            else:
                return VOCAB.LOCAL_TROUGH_MODERATE
        else:
            if width > 20:
                return VOCAB.ROUND_BOTTOM
            else:
                return VOCAB.LOCAL_TROUGH_STRONG

In [26]:
# ============================================================================
# 6. VOLATILITY REGIME DETECTOR
# ============================================================================

@dataclass
class VolatilityRegime:
    start: int
    end: int
    level: str  # 'low', 'normal', 'elevated', 'high'
    avg_vol: float
    label: int

class VolatilityRegimeDetector:
    """Detect volatility regimes"""
    
    def detect(self, x: torch.Tensor, features: Dict, 
               sample_idx: int = 0) -> List[VolatilityRegime]:
        """Detect volatility regimes in single sequence"""
        
        # Use rolling std
        if 'std_20' in features:
            vol = features['std_20'][sample_idx].cpu().numpy()
        else:
            return []
        
        L = len(vol)
        
        # Compute quantiles
        q25, q50, q75, q90 = np.percentile(vol, [25, 50, 75, 90])
        
        # Classify each point
        vol_levels = np.zeros(L, dtype=int)
        vol_levels[vol <= q25] = 0  # low
        vol_levels[(vol > q25) & (vol <= q50)] = 1  # normal
        vol_levels[(vol > q50) & (vol <= q75)] = 2  # elevated
        vol_levels[(vol > q75) & (vol <= q90)] = 3  # high
        vol_levels[vol > q90] = 4  # spike
        
        # Find regime changes
        changes = np.where(np.diff(vol_levels) != 0)[0] + 1
        breakpoints = np.concatenate([[0], changes, [L]])
        
        regimes = []
        for i in range(len(breakpoints) - 1):
            start = breakpoints[i]
            end = breakpoints[i + 1]
            
            if end - start < 5:  # Skip very short regimes
                continue
            
            level_code = vol_levels[start]
            avg_vol = vol[start:end].mean()
            
            # Map to label
            if level_code == 0:
                label = VOCAB.LOW_VOLATILITY
                level = 'low'
            elif level_code == 1:
                label = VOCAB.NORMAL_VOLATILITY
                level = 'normal'
            elif level_code == 2:
                label = VOCAB.ELEVATED_VOLATILITY
                level = 'elevated'
            elif level_code == 3:
                label = VOCAB.HIGH_VOLATILITY
                level = 'high'
            else:
                label = VOCAB.VOLATILITY_SPIKE
                level = 'spike'
            
            regimes.append(VolatilityRegime(
                start=start,
                end=end,
                level=level,
                avg_vol=float(avg_vol),
                label=label
            ))
        
        return regimes


In [27]:
# ============================================================================
# 7. CHANGE POINT DETECTOR (CUSUM-based)
# ============================================================================

@dataclass
class ChangePoint:
    index: int
    type: str  # 'mean_shift_up', 'mean_shift_down', etc.
    magnitude: float
    label: int

class ChangePointDetector:
    """Detect change points using CUSUM"""
    
    def __init__(self, threshold=5.0, drift=0.5):
        self.threshold = threshold
        self.drift = drift
    
    def detect(self, x: torch.Tensor, sample_idx: int = 0) -> List[ChangePoint]:
        """Detect change points in single sequence"""
        x_np = x.cpu().numpy()
        L = len(x_np)
        
        # Normalize
        x_norm = (x_np - x_np.mean()) / (x_np.std() + 1e-8)
        
        # CUSUM for mean shifts
        cusum_pos = np.zeros(L)
        cusum_neg = np.zeros(L)
        
        for i in range(1, L):
            cusum_pos[i] = max(0, cusum_pos[i-1] + x_norm[i] - self.drift)
            cusum_neg[i] = min(0, cusum_neg[i-1] + x_norm[i] + self.drift)
        
        # Find threshold crossings
        change_points = []
        
        # Upward shifts
        up_crossings = np.where(cusum_pos > self.threshold)[0]
        if len(up_crossings) > 0:
            # Group nearby crossings
            groups = self._group_nearby(up_crossings, gap=10)
            for group in groups:
                idx = group[0]
                magnitude = cusum_pos[idx]
                change_points.append(ChangePoint(
                    index=int(idx),
                    type='mean_shift_up',
                    magnitude=float(magnitude),
                    label=VOCAB.MEAN_SHIFT_UP
                ))
        
        # Downward shifts
        down_crossings = np.where(cusum_neg < -self.threshold)[0]
        if len(down_crossings) > 0:
            groups = self._group_nearby(down_crossings, gap=10)
            for group in groups:
                idx = group[0]
                magnitude = abs(cusum_neg[idx])
                change_points.append(ChangePoint(
                    index=int(idx),
                    type='mean_shift_down',
                    magnitude=float(magnitude),
                    label=VOCAB.MEAN_SHIFT_DOWN
                ))
        
        # Sort by index
        change_points.sort(key=lambda cp: cp.index)
        
        return change_points
    
    def _group_nearby(self, indices: np.ndarray, gap: int = 10) -> List[List[int]]:
        """Group indices that are close together"""
        if len(indices) == 0:
            return []
        
        groups = []
        current_group = [indices[0]]
        
        for idx in indices[1:]:
            if idx - current_group[-1] <= gap:
                current_group.append(idx)
            else:
                groups.append(current_group)
                current_group = [idx]
        
        groups.append(current_group)
        return groups

In [28]:
# ============================================================================
# 8. COMPREHENSIVE EVENT DATASET
# ============================================================================

@dataclass
class ComprehensiveAnnotation:
    """Complete annotation for one sequence"""
    sequence: torch.Tensor  # [L]
    step_labels: torch.Tensor  # [L] step-wise labels
    trend_segments: List[TrendSegment]
    peaks_troughs: List[PeakTroughEvent]
    volatility_regimes: List[VolatilityRegime]
    change_points: List[ChangePoint]
    
    def to_event_sequence(self, max_events: int = 100) -> List[Dict]:
        """Convert to flat event list sorted by start index"""
        events = []
        
        # Add trends
        for seg in self.trend_segments:
            events.append({
                'start': seg.start,
                'end': seg.end,
                'type': 'trend',
                'label': seg.label,
                'label_name': VOCAB.id_to_label(seg.label),
                'metadata': {
                    'direction': seg.direction,
                    'slope': seg.slope,
                    'strength': seg.strength
                }
            })
        
        # Add peaks/troughs
        for pt in self.peaks_troughs:
            events.append({
                'start': pt.index,
                'end': pt.index,
                'type': 'peak_trough',
                'label': pt.label,
                'label_name': VOCAB.id_to_label(pt.label),
                'metadata': {
                    'peak_type': pt.type,
                    'prominence': pt.prominence
                }
            })
        
        # Add volatility regimes
        for vr in self.volatility_regimes:
            events.append({
                'start': vr.start,
                'end': vr.end,
                'type': 'volatility',
                'label': vr.label,
                'label_name': VOCAB.id_to_label(vr.label),
                'metadata': {
                    'level': vr.level,
                    'avg_vol': vr.avg_vol
                }
            })
        
        # Add change points
        for cp in self.change_points:
            events.append({
                'start': cp.index,
                'end': cp.index,
                'type': 'change_point',
                'label': cp.label,
                'label_name': VOCAB.id_to_label(cp.label),
                'metadata': {
                    'cp_type': cp.type,
                    'magnitude': cp.magnitude
                }
            })
        
        # Sort by start index
        events.sort(key=lambda e: e['start'])
        
        # Limit to max_events
        return events[:max_events]
    
    def to_text_description(self) -> str:
        """Generate natural language description"""
        events = self.to_event_sequence()
        
        parts = []
        parts.append(f"Sequence length: {len(self.sequence)}")
        parts.append(f"Total events detected: {len(events)}")
        
        for event in events:
            if event['start'] == event['end']:
                parts.append(
                    f"[{event['start']}] {event['label_name']} "
                    f"(type={event['type']})"
                )
            else:
                parts.append(
                    f"[{event['start']}-{event['end']}] {event['label_name']} "
                    f"(type={event['type']})"
                )
        
        return " | ".join(parts)

class ComprehensiveEventDataset(Dataset):
    """Complete dataset with all event types"""
    
    def __init__(self, x: torch.Tensor, verbose: bool = True):
        """
        x: [B, L] time series tensor
        """
        super().__init__()
        if x.dim() != 2:
            raise ValueError("Expected x with shape [B, L]")
        
        self.x = x
        B, L = x.shape
        
        if verbose:
            print(f"Processing {B} sequences of length {L}...")
        
        # Initialize detectors
        self.feature_extractor = MultiScaleFeatureExtractor()
        self.step_encoder = StepWiseEncoder()
        self.trend_detector = TrendSegmentDetector()
        self.peak_detector = PeakTroughDetector()
        self.vol_detector = VolatilityRegimeDetector()
        self.cp_detector = ChangePointDetector()
        
        # Extract features (batch-wise)
        if verbose:
            print("Extracting multi-scale features...")
        self.features = self.feature_extractor.extract_features(x)
        
        # Encode step labels (batch-wise)
        if verbose:
            print("Encoding step-wise labels...")
        self.step_labels = self.step_encoder.encode(x, self.features)
        
        # Process each sequence for higher-level events
        if verbose:
            print("Detecting trends, peaks, volatility, change points...")
        
        self.annotations = []
        for i in range(B):
            if verbose and i % 100 == 0:
                print(f"  Processing sequence {i}/{B}...")
            
            trends = self.trend_detector.detect(x[i], self.features, i)
            peaks = self.peak_detector.detect(x[i], i)
            vol_regimes = self.vol_detector.detect(x[i], self.features, i)
            change_pts = self.cp_detector.detect(x[i], i)
            
            annotation = ComprehensiveAnnotation(
                sequence=x[i],
                step_labels=self.step_labels[i],
                trend_segments=trends,
                peaks_troughs=peaks,
                volatility_regimes=vol_regimes,
                change_points=change_pts
            )
            
            self.annotations.append(annotation)
        
        if verbose:
            print(f"✓ Dataset ready with {len(self.annotations)} annotated sequences")
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        """Return comprehensive annotation"""
        return self.annotations[idx]
    
    def get_statistics(self):
        """Compute dataset statistics"""
        total_trends = sum(len(ann.trend_segments) for ann in self.annotations)
        total_peaks = sum(len(ann.peaks_troughs) for ann in self.annotations)
        total_vol = sum(len(ann.volatility_regimes) for ann in self.annotations)
        total_cp = sum(len(ann.change_points) for ann in self.annotations)
        
        avg_trends = total_trends / len(self.annotations)
        avg_peaks = total_peaks / len(self.annotations)
        avg_vol = total_vol / len(self.annotations)
        avg_cp = total_cp / len(self.annotations)
        
        return {
            'num_sequences': len(self.annotations),
            'sequence_length': len(self.annotations[0].sequence),
            'avg_trend_segments': avg_trends,
            'avg_peaks_troughs': avg_peaks,
            'avg_volatility_regimes': avg_vol,
            'avg_change_points': avg_cp,
            'avg_total_events': avg_trends + avg_peaks + avg_vol + avg_cp,
            'vocab_size': VOCAB.get_vocab_size()
        }

In [29]:
# ============================================================================
# 9. TEXT CORPUS GENERATOR FOR LM TRAINING
# ============================================================================

class TextCorpusGenerator:
    """Generate text corpus for language model training"""
    
    @staticmethod
    def generate_structured_text(annotation: ComprehensiveAnnotation) -> str:
        """Structured format (token-efficient)"""
        events = annotation.to_event_sequence()
        
        tokens = []
        for e in events:
            if e['start'] == e['end']:
                tokens.append(f"[{e['start']}]{e['label_name']}")
            else:
                tokens.append(f"[{e['start']}-{e['end']}]{e['label_name']}")
        
        return " ".join(tokens)
    
    @staticmethod
    def generate_hybrid_text(annotation: ComprehensiveAnnotation) -> str:
        """Hybrid: structured + metadata"""
        events = annotation.to_event_sequence()
        
        parts = []
        for e in events:
            span = f"[{e['start']}-{e['end']}]" if e['start'] != e['end'] else f"[{e['start']}]"
            metadata_str = " ".join(f"{k}={v}" for k, v in e['metadata'].items())
            parts.append(f"{span} {e['label_name']} ({metadata_str})")
        
        return " | ".join(parts)
    
    @staticmethod
    def generate_narrative_text(annotation: ComprehensiveAnnotation) -> str:
        """Natural language narrative"""
        events = annotation.to_event_sequence()
        L = len(annotation.sequence)
        
        sentences = [f"Time series of length {L}."]
        
        # Group by type
        trends = [e for e in events if e['type'] == 'trend']
        peaks = [e for e in events if e['type'] == 'peak_trough']
        
        if trends:
            sentences.append(f"Contains {len(trends)} trend segments:")
            for t in trends[:3]:  # Limit verbosity
                sentences.append(
                    f"  From {t['start']} to {t['end']}, "
                    f"{t['label_name'].lower().replace('_', ' ')} observed."
                )
        
        if peaks:
            sentences.append(f"Notable peaks/troughs: {len(peaks)} detected.")
        
        return " ".join(sentences)


In [30]:
# ============================================================================
# 10. EXAMPLE USAGE
# ============================================================================

if __name__ == "__main__":
    # Example: Process batch of EEG-like signals
    print("=" * 80)
    print("COMPREHENSIVE TIME SERIES EVENT LABELING SYSTEM")
    print("=" * 80)
    
    # Generate synthetic data
    B = 100  # batch size
    L = 336  # sequence length
    
    print(f"\nGenerating {B} synthetic sequences of length {L}...")
    
    # Create realistic time series (trending + noise + spikes)
    torch.manual_seed(42)
    t = torch.linspace(0, 4*np.pi, L)
    x = torch.zeros(B, L)
    
    for i in range(B):
        # Base trend
        trend = 0.5 * torch.sin(t / 2) + 0.1 * t
        
        # Add volatility clusters
        vol = 0.1 + 0.2 * (torch.sin(3 * t) > 0).float()
        noise = torch.randn(L) * vol
        
        # Add occasional spikes
        spike_indices = torch.randint(0, L, (3,))
        spikes = torch.zeros(L)
        spikes[spike_indices] = torch.randn(3) * 2
        
        x[i] = trend + noise + spikes
    
    # Create comprehensive dataset
    dataset = ComprehensiveEventDataset(x, verbose=True)
    
    # Get statistics
    print("\n" + "=" * 80)
    print("DATASET STATISTICS")
    print("=" * 80)
    stats = dataset.get_statistics()
    for key, value in stats.items():
        print(f"{key:.<40} {value:.2f}")
    
    # Example: Get annotation for first sequence
    print("\n" + "=" * 80)
    print("EXAMPLE ANNOTATION (First Sequence)")
    print("=" * 80)
    
    ann = dataset[0]
    
    print(f"\nStep labels (first 20): {ann.step_labels[:20].tolist()}")
    print(f"  -> {[VOCAB.id_to_label(int(l)) for l in ann.step_labels[:20]]}")
    
    print(f"\nTrend segments detected: {len(ann.trend_segments)}")
    for i, seg in enumerate(ann.trend_segments[:5]):
        print(f"  {i+1}. [{seg.start}-{seg.end}] {VOCAB.id_to_label(seg.label)} "
              f"(slope={seg.slope:.3f})")
    
    print(f"\nPeaks/Troughs detected: {len(ann.peaks_troughs)}")
    for i, pt in enumerate(ann.peaks_troughs[:5]):
        print(f"  {i+1}. [{pt.index}] {VOCAB.id_to_label(pt.label)} "
              f"(prominence={pt.prominence:.3f})")
    
    print(f"\nVolatility regimes: {len(ann.volatility_regimes)}")
    for i, vr in enumerate(ann.volatility_regimes):
        print(f"  {i+1}. [{vr.start}-{vr.end}] {VOCAB.id_to_label(vr.label)} "
              f"(avg_vol={vr.avg_vol:.3f})")
    
    print(f"\nChange points: {len(ann.change_points)}")
    for i, cp in enumerate(ann.change_points):
        print(f"  {i+1}. [{cp.index}] {VOCAB.id_to_label(cp.label)} "
              f"(magnitude={cp.magnitude:.3f})")
    
    # Generate text descriptions
    print("\n" + "=" * 80)
    print("TEXT GENERATION FOR LM TRAINING")
    print("=" * 80)
    
    text_gen = TextCorpusGenerator()
    
    print("\n1. STRUCTURED FORMAT:")
    print(text_gen.generate_structured_text(ann)[:500] + "...")
    
    print("\n2. HYBRID FORMAT:")
    print(text_gen.generate_hybrid_text(ann)[:500] + "...")
    
    print("\n3. NARRATIVE FORMAT:")
    print(text_gen.generate_narrative_text(ann)[:500] + "...")
    
    # Token count estimation
    print("\n" + "=" * 80)
    print("TOKEN COUNT ESTIMATION")
    print("=" * 80)
    
    structured = text_gen.generate_structured_text(ann)
    hybrid = text_gen.generate_hybrid_text(ann)
    
    # Simple token count (split by whitespace)
    print(f"Structured format: ~{len(structured.split())} tokens")
    print(f"Hybrid format:     ~{len(hybrid.split())} tokens")
    
    print("\n" + "=" * 80)
    print("✓ System demonstration complete!")
    print("=" * 80)

COMPREHENSIVE TIME SERIES EVENT LABELING SYSTEM

Generating 100 synthetic sequences of length 336...
Processing 100 sequences of length 336...
Extracting multi-scale features...
Encoding step-wise labels...
Detecting trends, peaks, volatility, change points...
  Processing sequence 0/100...
✓ Dataset ready with 100 annotated sequences

DATASET STATISTICS
num_sequences........................... 100.00
sequence_length......................... 336.00
avg_trend_segments...................... 35.34
avg_peaks_troughs....................... 54.26
avg_volatility_regimes.................. 27.10
avg_change_points....................... 4.17
avg_total_events........................ 120.87
vocab_size.............................. 121.00

EXAMPLE ANNOTATION (First Sequence)

Step labels (first 20): [0, 7, 11, 19, 16, 13, 7, 13, 7, 8, 13, 12, 7, 5, 9, 8, 7, 13, 10, 7]
  -> ['PAD', 'UP_LARGE', 'DOWN_MEDIUM', 'SPIKE_DOWN_EXTREME', 'SPIKE_UP_EXTREME', 'DOWN_HUGE', 'UP_LARGE', 'DOWN_HUGE', 'UP_LARGE', 

In [31]:
# ============================================================================
# HIERARCHICAL EVENT SYSTEM (Add to existing code)
# ============================================================================

from typing import List, Dict, Optional, Set
from dataclasses import dataclass, field
from enum import IntEnum

class EventScale(IntEnum):
    """Hierarchical scale levels"""
    MICRO = 1      # Single points, spikes (1-5 steps)
    MINI = 2       # Very short segments (5-15 steps)
    MESO = 3       # Medium segments (15-50 steps)
    MACRO = 4      # Long segments (50-150 steps)
    GLOBAL = 5     # Full sequence level (150+ steps)

@dataclass
class HierarchicalEvent:
    """Event node in hierarchy tree"""
    start: int
    end: int
    label: int
    label_name: str
    scale: EventScale
    event_type: str  # 'trend', 'volatility', 'peak', 'changepoint', etc.
    confidence: float
    metadata: Dict
    
    # Hierarchical relationships
    parent: Optional['HierarchicalEvent'] = None
    children: List['HierarchicalEvent'] = field(default_factory=list)
    
    @property
    def duration(self) -> int:
        return self.end - start + 1
    
    @property
    def depth(self) -> int:
        """Depth in hierarchy (0 = root)"""
        depth = 0
        node = self.parent
        while node is not None:
            depth += 1
            node = node.parent
        return depth
    
    def contains(self, other: 'HierarchicalEvent') -> bool:
        """Check if this event contains another"""
        return (self.start <= other.start and 
                self.end >= other.end and
                self != other)
    
    def overlaps(self, other: 'HierarchicalEvent') -> bool:
        """Check if events overlap (but neither contains the other)"""
        if self.contains(other) or other.contains(self):
            return False
        return not (self.end < other.start or self.start > other.end)
    
    def __repr__(self):
        indent = "  " * self.depth
        children_info = f" ({len(self.children)} children)" if self.children else ""
        return (f"{indent}[{self.start}-{self.end}] {self.label_name} "
                f"(scale={self.scale.name}){children_info}")

# ============================================================================
# HIERARCHICAL EVENT BUILDER
# ============================================================================

class HierarchicalEventBuilder:
    """Build hierarchical event tree from flat event list"""
    
    def __init__(self):
        self.events = []
    
    def add_event(self, start: int, end: int, label: int, 
                  event_type: str, confidence: float = 1.0, 
                  metadata: Optional[Dict] = None):
        """Add event to the collection"""
        duration = end - start + 1
        
        # Determine scale based on duration
        if duration <= 5:
            scale = EventScale.MICRO
        elif duration <= 15:
            scale = EventScale.MINI
        elif duration <= 50:
            scale = EventScale.MESO
        elif duration <= 150:
            scale = EventScale.MACRO
        else:
            scale = EventScale.GLOBAL
        
        event = HierarchicalEvent(
            start=start,
            end=end,
            label=label,
            label_name=VOCAB.id_to_label(label),
            scale=scale,
            event_type=event_type,
            confidence=confidence,
            metadata=metadata or {}
        )
        
        self.events.append(event)
    
    def build_hierarchy(self) -> List[HierarchicalEvent]:
        """Build parent-child relationships"""
        
        # Sort by scale (largest first), then by start position
        sorted_events = sorted(
            self.events,
            key=lambda e: (-e.scale, e.start, -e.duration)
        )
        
        # Build hierarchy tree
        roots = []
        
        for event in sorted_events:
            # Find the smallest containing event as parent
            parent = self._find_parent(event, sorted_events)
            
            if parent is None:
                # This is a root node
                roots.append(event)
            else:
                # Add as child to parent
                event.parent = parent
                parent.children.append(event)
        
        # Sort children by start position within each parent
        self._sort_children(roots)
        
        return roots
    
    def _find_parent(self, event: HierarchicalEvent, 
                     all_events: List[HierarchicalEvent]) -> Optional[HierarchicalEvent]:
        """Find smallest event that contains this event"""
        candidates = []
        
        for potential_parent in all_events:
            if potential_parent == event:
                continue
            
            # Must be larger scale
            if potential_parent.scale <= event.scale:
                continue
            
            # Must contain the event
            if potential_parent.contains(event):
                candidates.append(potential_parent)
        
        if not candidates:
            return None
        
        # Return the smallest containing event (most specific parent)
        return min(candidates, key=lambda e: e.duration)
    
    def _sort_children(self, nodes: List[HierarchicalEvent]):
        """Recursively sort children by start position"""
        for node in nodes:
            if node.children:
                node.children.sort(key=lambda c: (c.start, -c.duration))
                self._sort_children(node.children)
    
    def get_flat_list(self, roots: List[HierarchicalEvent]) -> List[HierarchicalEvent]:
        """Get flattened list of all events (depth-first traversal)"""
        result = []
        
        def traverse(node: HierarchicalEvent):
            result.append(node)
            for child in node.children:
                traverse(child)
        
        for root in roots:
            traverse(root)
        
        return result

# ============================================================================
# ENHANCED COMPREHENSIVE ANNOTATION WITH HIERARCHY
# ============================================================================

@dataclass
class HierarchicalAnnotation:
    """Complete hierarchical annotation for one sequence"""
    sequence: torch.Tensor  # [L]
    step_labels: torch.Tensor  # [L] step-wise labels
    
    # Hierarchical event tree
    event_roots: List[HierarchicalEvent]
    all_events: List[HierarchicalEvent]  # Flattened
    
    def print_hierarchy(self, max_depth: int = 10):
        """Pretty print hierarchy"""
        def print_tree(node: HierarchicalEvent, depth: int = 0):
            if depth > max_depth:
                return
            print(node)
            for child in node.children:
                print_tree(child, depth + 1)
        
        print(f"\nHierarchical Event Tree (Total: {len(self.all_events)} events)")
        print("=" * 80)
        for root in self.event_roots:
            print_tree(root)
    
    def get_events_at_scale(self, scale: EventScale) -> List[HierarchicalEvent]:
        """Get all events at specific scale"""
        return [e for e in self.all_events if e.scale == scale]
    
    def get_events_in_range(self, start: int, end: int) -> List[HierarchicalEvent]:
        """Get all events that overlap with range [start, end]"""
        result = []
        for event in self.all_events:
            if not (event.end < start or event.start > end):
                result.append(event)
        return result
    
    def to_hierarchical_text(self, format: str = 'structured') -> str:
        """Generate hierarchical text representation"""
        if format == 'structured':
            return self._structured_hierarchical_text()
        elif format == 'indented':
            return self._indented_hierarchical_text()
        elif format == 'narrative':
            return self._narrative_hierarchical_text()
        else:
            raise ValueError(f"Unknown format: {format}")
    
    def _structured_hierarchical_text(self) -> str:
        """Compact structured format with depth indicators"""
        parts = []
        
        def traverse(node: HierarchicalEvent):
            depth_marker = ">" * node.depth
            parts.append(
                f"{depth_marker}[{node.start}-{node.end}]"
                f"{node.label_name}"
                f"@{node.scale.name}"
            )
            for child in node.children:
                traverse(child)
        
        for root in self.event_roots:
            traverse(root)
        
        return " ".join(parts)
    
    def _indented_hierarchical_text(self) -> str:
        """Human-readable indented format"""
        lines = []
        
        def traverse(node: HierarchicalEvent):
            indent = "  " * node.depth
            lines.append(
                f"{indent}[{node.start:03d}-{node.end:03d}] "
                f"{node.label_name} "
                f"(scale={node.scale.name}, conf={node.confidence:.2f})"
            )
            for child in node.children:
                traverse(child)
        
        for root in self.event_roots:
            traverse(root)
        
        return "\n".join(lines)
    
    def _narrative_hierarchical_text(self) -> str:
        """Natural language with hierarchical context"""
        sentences = []
        
        # Start with macro view
        macro_events = self.get_events_at_scale(EventScale.MACRO)
        global_events = self.get_events_at_scale(EventScale.GLOBAL)
        
        if global_events:
            sentences.append(
                f"Overall, the sequence exhibits {global_events[0].label_name.lower().replace('_', ' ')}."
            )
        
        if macro_events:
            sentences.append(f"At the macro level, {len(macro_events)} major segments:")
            for event in macro_events[:3]:
                sentences.append(
                    f"  From {event.start} to {event.end}, "
                    f"{event.label_name.lower().replace('_', ' ')}"
                )
                
                # Mention nested events
                if event.children:
                    nested_types = set(c.event_type for c in event.children)
                    sentences.append(
                        f"    (contains {len(event.children)} nested events: "
                        f"{', '.join(nested_types)})"
                    )
        
        return " ".join(sentences)

# ============================================================================
# ENHANCED DATASET WITH HIERARCHY
# ============================================================================

class HierarchicalEventDataset(Dataset):
    """Dataset with full hierarchical event structure"""
    
    def __init__(self, x: torch.Tensor, verbose: bool = True):
        """
        x: [B, L] time series tensor
        """
        super().__init__()
        if x.dim() != 2:
            raise ValueError("Expected x with shape [B, L]")
        
        self.x = x
        B, L = x.shape
        
        if verbose:
            print(f"Processing {B} sequences of length {L} with hierarchical structure...")
        
        # Initialize all detectors (reuse from previous implementation)
        self.feature_extractor = MultiScaleFeatureExtractor()
        self.step_encoder = StepWiseEncoder()
        self.trend_detector = TrendSegmentDetector()
        self.peak_detector = PeakTroughDetector()
        self.vol_detector = VolatilityRegimeDetector()
        self.cp_detector = ChangePointDetector()
        
        # Extract features
        if verbose:
            print("Extracting features...")
        self.features = self.feature_extractor.extract_features(x)
        
        # Encode step labels
        if verbose:
            print("Encoding step labels...")
        self.step_labels = self.step_encoder.encode(x, self.features)
        
        # Build hierarchical annotations
        if verbose:
            print("Building hierarchical event structure...")
        
        self.annotations = []
        for i in range(B):
            if verbose and i % 100 == 0:
                print(f"  Sequence {i}/{B}...")
            
            annotation = self._build_hierarchical_annotation(i)
            self.annotations.append(annotation)
        
        if verbose:
            print(f"✓ Dataset ready with hierarchical structure")
            self._print_statistics()
    
    def _build_hierarchical_annotation(self, idx: int) -> HierarchicalAnnotation:
        """Build complete hierarchical annotation for one sequence"""
        
        builder = HierarchicalEventBuilder()
        
        # 1. Detect all events (as before)
        trends = self.trend_detector.detect(self.x[idx], self.features, idx)
        peaks = self.peak_detector.detect(self.x[idx], idx)
        vol_regimes = self.vol_detector.detect(self.x[idx], self.features, idx)
        change_pts = self.cp_detector.detect(self.x[idx], idx)
        
        # 2. Add trend segments to builder
        for seg in trends:
            builder.add_event(
                start=seg.start,
                end=seg.end,
                label=seg.label,
                event_type='trend',
                confidence=0.9,
                metadata={
                    'direction': seg.direction,
                    'slope': seg.slope,
                    'strength': seg.strength
                }
            )
        
        # 3. Add peaks/troughs
        for pt in peaks:
            builder.add_event(
                start=pt.index,
                end=pt.index,
                label=pt.label,
                event_type='peak_trough',
                confidence=0.85,
                metadata={
                    'peak_type': pt.type,
                    'prominence': pt.prominence
                }
            )
        
        # 4. Add volatility regimes
        for vr in vol_regimes:
            builder.add_event(
                start=vr.start,
                end=vr.end,
                label=vr.label,
                event_type='volatility',
                confidence=0.8,
                metadata={
                    'level': vr.level,
                    'avg_vol': vr.avg_vol
                }
            )
        
        # 5. Add change points
        for cp in change_pts:
            builder.add_event(
                start=cp.index,
                end=cp.index,
                label=cp.label,
                event_type='changepoint',
                confidence=0.75,
                metadata={
                    'cp_type': cp.type,
                    'magnitude': cp.magnitude
                }
            )
        
        # 6. Add global regime (macro-level characterization)
        L = len(self.x[idx])
        global_regime = self._classify_global_regime(idx, L)
        builder.add_event(
            start=0,
            end=L-1,
            label=global_regime,
            event_type='global_regime',
            confidence=0.7,
            metadata={'scope': 'global'}
        )
        
        # 7. Build hierarchy
        roots = builder.build_hierarchy()
        all_events = builder.get_flat_list(roots)
        
        return HierarchicalAnnotation(
            sequence=self.x[idx],
            step_labels=self.step_labels[idx],
            event_roots=roots,
            all_events=all_events
        )
    
    def _classify_global_regime(self, idx: int, length: int) -> int:
        """Classify overall regime for entire sequence"""
        # Use net slope across entire sequence
        if 'slope_100' in self.features:
            avg_slope = self.features['slope_100'][idx].mean().item()
        else:
            avg_slope = 0
        
        # Use average volatility
        if 'std_20' in self.features:
            avg_vol = self.features['std_20'][idx].mean().item()
        else:
            avg_vol = 0.5
        
        # Classify
        if avg_vol > 1.5:
            return VOCAB.VOLATILE_REGIME
        elif avg_slope > 0.1:
            return VOCAB.BULLISH_REGIME
        elif avg_slope < -0.1:
            return VOCAB.BEARISH_REGIME
        else:
            return VOCAB.SIDEWAYS_REGIME
    
    def _print_statistics(self):
        """Print hierarchical statistics"""
        print("\n" + "=" * 80)
        print("HIERARCHICAL STATISTICS")
        print("=" * 80)
        
        # Count events by scale
        scale_counts = {scale: 0 for scale in EventScale}
        total_events = 0
        max_depth = 0
        
        for ann in self.annotations:
            total_events += len(ann.all_events)
            for event in ann.all_events:
                scale_counts[event.scale] += 1
                max_depth = max(max_depth, event.depth)
        
        avg_events = total_events / len(self.annotations)
        
        print(f"Sequences: {len(self.annotations)}")
        print(f"Avg events per sequence: {avg_events:.1f}")
        print(f"Max hierarchy depth: {max_depth}")
        print(f"\nEvents by scale:")
        for scale in EventScale:
            count = scale_counts[scale]
            avg = count / len(self.annotations)
            print(f"  {scale.name:.<20} {avg:>6.1f} per sequence")
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        return self.annotations[idx]

# ============================================================================
# TEXT GENERATION WITH HIERARCHY
# ============================================================================

class HierarchicalTextGenerator:
    """Generate training text that preserves hierarchical structure"""
    
    @staticmethod
    def generate_flat_sequential(ann: HierarchicalAnnotation) -> str:
        """Flat list sorted by start (loses hierarchy)"""
        events = sorted(ann.all_events, key=lambda e: e.start)
        
        tokens = []
        for e in events:
            tokens.append(f"[{e.start}-{e.end}]{e.label_name}")
        
        return " ".join(tokens)
    
    @staticmethod
    def generate_depth_marked(ann: HierarchicalAnnotation) -> str:
        """Include depth markers to indicate nesting"""
        parts = []
        
        def traverse(node: HierarchicalEvent):
            depth_str = ">" * node.depth
            parts.append(
                f"{depth_str}[{node.start}-{node.end}]{node.label_name}"
            )
            for child in node.children:
                traverse(child)
        
        for root in ann.event_roots:
            traverse(root)
        
        return " ".join(parts)
    
    @staticmethod
    def generate_xml_style(ann: HierarchicalAnnotation) -> str:
        """XML-like nested structure"""
        lines = []
        
        def traverse(node: HierarchicalEvent, depth: int = 0):
            indent = "  " * depth
            if node.children:
                lines.append(f"{indent}<event type='{node.label_name}' span='{node.start}-{node.end}'>")
                for child in node.children:
                    traverse(child, depth + 1)
                lines.append(f"{indent}</event>")
            else:
                lines.append(
                    f"{indent}<event type='{node.label_name}' "
                    f"span='{node.start}-{node.end}' />"
                )
        
        for root in ann.event_roots:
            traverse(root)
        
        return "\n".join(lines)
    
    @staticmethod
    def generate_json_style(ann: HierarchicalAnnotation) -> str:
        """JSON-like hierarchical structure"""
        import json
        
        def event_to_dict(node: HierarchicalEvent) -> Dict:
            result = {
                'span': [node.start, node.end],
                'label': node.label_name,
                'scale': node.scale.name,
                'confidence': node.confidence
            }
            if node.children:
                result['children'] = [event_to_dict(child) for child in node.children]
            return result
        
        tree = [event_to_dict(root) for root in ann.event_roots]
        return json.dumps(tree, separators=(',', ':'))
    
    @staticmethod
    def generate_narrative_with_context(ann: HierarchicalAnnotation) -> str:
        """Natural language that mentions parent context"""
        sentences = []
        
        def traverse(node: HierarchicalEvent):
            # Describe the event
            if node.start == node.end:
                span_desc = f"At position {node.start}"
            else:
                span_desc = f"From {node.start} to {node.end}"
            
            event_desc = node.label_name.lower().replace('_', ' ')
            
            # Add parent context if exists
            if node.parent:
                parent_desc = node.parent.label_name.lower().replace('_', ' ')
                sentences.append(
                    f"{span_desc}, {event_desc} occurs "
                    f"(within {parent_desc} [{node.parent.start}-{node.parent.end}])."
                )
            else:
                sentences.append(f"{span_desc}, {event_desc}.")
            
            # Recurse
            for child in node.children:
                traverse(child)
        
        for root in ann.event_roots:
            traverse(root)
        
        return " ".join(sentences)

# ============================================================================
# UPDATED EXAMPLE WITH HIERARCHY
# ============================================================================

if __name__ == "__main__":
    print("=" * 80)
    print("HIERARCHICAL TIME SERIES EVENT LABELING SYSTEM")
    print("=" * 80)
    
    # Generate test data
    B = 50
    L = 336
    
    print(f"\nGenerating {B} sequences of length {L}...")
    torch.manual_seed(42)
    
    t = torch.linspace(0, 4*np.pi, L)
    x = torch.zeros(B, L)
    
    for i in range(B):
        # Multi-scale trend
        trend = 0.5 * torch.sin(t / 2) + 0.1 * t
        
        # Volatility clusters
        vol = 0.1 + 0.2 * (torch.sin(3 * t) > 0).float()
        noise = torch.randn(L) * vol
        
        # Spikes
        spike_indices = torch.randint(0, L, (3,))
        spikes = torch.zeros(L)
        spikes[spike_indices] = torch.randn(3) * 2
        
        # Local corrections (create nested structure)
        if i % 3 == 0:
            # Add dip in middle of uptrend
            x[i, 150:180] = x[i, 150:180] - 0.5
        
        x[i] = trend + noise + spikes
    
    # Create hierarchical dataset
    dataset = HierarchicalEventDataset(x, verbose=True)
    
    # Example annotation
    print("\n" + "=" * 80)
    print("HIERARCHICAL STRUCTURE EXAMPLE")
    print("=" * 80)
    
    ann = dataset[0]
    ann.print_hierarchy(max_depth=3)
    
    # Show events by scale
    print("\n" + "=" * 80)
    print("EVENTS BY SCALE")
    print("=" * 80)
    for scale in EventScale:
        events = ann.get_events_at_scale(scale)
        print(f"\n{scale.name} ({len(events)} events):")
        for e in events[:5]:
            print(f"  [{e.start:03d}-{e.end:03d}] {e.label_name}")
    
    # Generate different text formats
    print("\n" + "=" * 80)
    print("TEXT GENERATION FORMATS")
    print("=" * 80)
    
    text_gen = HierarchicalTextGenerator()
    
    print("\n1. DEPTH-MARKED FORMAT:")
    print(text_gen.generate_depth_marked(ann)[:400] + "...")
    
    print("\n2. NARRATIVE WITH CONTEXT:")
    narrative = text_gen.generate_narrative_with_context(ann)
    print(narrative[:400] + "...")
    
    print("\n3. JSON-STYLE (first 500 chars):")
    print(text_gen.generate_json_style(ann)[:500] + "...")
    
    # Token count comparison
    print("\n" + "=" * 80)
    print("TOKEN COUNTS BY FORMAT")
    print("=" * 80)
    
    formats = {
        'Flat sequential': text_gen.generate_flat_sequential(ann),
        'Depth-marked': text_gen.generate_depth_marked(ann),
        'Narrative': text_gen.generate_narrative_with_context(ann),
        'JSON-style': text_gen.generate_json_style(ann)
    }
    
    for name, text in formats.items():
        tokens = len(text.split())
        chars = len(text)
        print(f"{name:.<30} {tokens:>6} tokens, {chars:>6} chars")
    
    print("\n" + "=" * 80)
    print("✓ Hierarchical system complete!")
    print("=" * 80)


HIERARCHICAL TIME SERIES EVENT LABELING SYSTEM

Generating 50 sequences of length 336...
Processing 50 sequences of length 336 with hierarchical structure...
Extracting features...
Encoding step labels...
Building hierarchical event structure...
  Sequence 0/50...


NameError: name 'start' is not defined

In [ ]:
"""
## 🎯 **What's New in This Version**

### **1. True Hierarchical Structure** ✅
```
[0-335] SIDEWAYS_REGIME (GLOBAL)
  ├─ [0-120] UPTREND_LONG_MODERATE (MACRO)
  │   ├─ [30-45] DOWNTREND_SHORT_WEAK (MESO)  ← NESTED!
  │   │   └─ [38] SPIKE_DOWN_STRONG (MICRO)
  │   └─ [50-55] VOLATILITY_SPIKE (MINI)
  ├─ [121-200] FLAT_LONG (MACRO)
  └─ [201-335] DOWNTREND_LONG_STRONG (MACRO)
      └─ [250-265] LOCAL_PEAK_MODERATE (MESO)
```

### **2. Event Scale Classification**
- **MICRO** (1-5 steps): Spikes, single peaks
- **MINI** (5-15 steps): Very short segments
- **MESO** (15-50 steps): Medium trends, local corrections
- **MACRO** (50-150 steps): Major trends
- **GLOBAL** (150+ steps): Overall regime

### **3. Parent-Child Relationships**
- Automatic detection of containment
- Smallest containing event becomes parent
- Children sorted by start position

### **4. Multiple Text Formats**

**Depth-Marked (Token-Efficient):**
```
[0-335]SIDEWAYS_REGIME@GLOBAL >[0-120]UPTREND_LONG_MODERATE@MACRO >>[30-45]DOWNTREND_SHORT_WEAK@MESO >>>[38]SPIKE_DOWN_STRONG@MICRO
```

**Narrative with Context:**
```
From 0 to 335, sideways regime. From 0 to 120, uptrend long moderate. 
From 30 to 45, downtrend short weak (within uptrend long moderate [0-120]). 
At position 38, spike down strong (within downtrend short weak [30-45]). """

"\n## 🎯 **What's New in This Version**\n\n### **1. True Hierarchical Structure** ✅\n```\n[0-335] SIDEWAYS_REGIME (GLOBAL)\n  ├─ [0-120] UPTREND_LONG_MODERATE (MACRO)\n  │   ├─ [30-45] DOWNTREND_SHORT_WEAK (MESO)  ← NESTED!\n  │   │   └─ [38] SPIKE_DOWN_STRONG (MICRO)\n  │   └─ [50-55] VOLATILITY_SPIKE (MINI)\n  ├─ [121-200] FLAT_LONG (MACRO)\n  └─ [201-335] DOWNTREND_LONG_STRONG (MACRO)\n      └─ [250-265] LOCAL_PEAK_MODERATE (MESO)\n```\n\n### **2. Event Scale Classification**\n- **MICRO** (1-5 steps): Spikes, single peaks\n- **MINI** (5-15 steps): Very short segments\n- **MESO** (15-50 steps): Medium trends, local corrections\n- **MACRO** (50-150 steps): Major trends\n- **GLOBAL** (150+ steps): Overall regime\n\n### **3. Parent-Child Relationships**\n- Automatic detection of containment\n- Smallest containing event becomes parent\n- Children sorted by start position\n\n### **4. Multiple Text Formats**\n\n**Depth-Marked (Token-Efficient):**\n```\n[0-335]SIDEWAYS_REGIME@GLOBAL >[0-120]

In [32]:
"""
COMPLETE HIERARCHICAL TIME SERIES EVENT LABELING SYSTEM
========================================================
Fixed and production-ready version
"""

import torch
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset
from dataclasses import dataclass, field
from typing import List, Dict, Tuple, Optional
from scipy import signal as scipy_signal
from enum import IntEnum
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# 1. VOCABULARY (same as before)
# ============================================================================

@dataclass
class EventVocabulary:
    """Complete event vocabulary"""
    PAD = 0
    MASK = 1
    FLAT = 2
    UP_SMALL = 3
    UP_MEDIUM = 4
    UP_LARGE = 5
    DOWN_SMALL = 6
    DOWN_MEDIUM = 7
    DOWN_LARGE = 8
    SPIKE_UP = 9
    SPIKE_DOWN = 10
    
    # Trends
    UPTREND_SHORT = 20
    UPTREND_MEDIUM = 21
    UPTREND_LONG = 22
    DOWNTREND_SHORT = 23
    DOWNTREND_MEDIUM = 24
    DOWNTREND_LONG = 25
    FLAT_SEGMENT = 26
    
    # Peaks
    LOCAL_PEAK = 30
    SHARP_PEAK = 31
    LOCAL_TROUGH = 32
    SHARP_TROUGH = 33
    
    # Volatility
    LOW_VOLATILITY = 40
    NORMAL_VOLATILITY = 41
    HIGH_VOLATILITY = 42
    VOLATILITY_SPIKE = 43
    
    # Change points
    MEAN_SHIFT_UP = 50
    MEAN_SHIFT_DOWN = 51
    
    # Regimes
    BULLISH_REGIME = 60
    BEARISH_REGIME = 61
    SIDEWAYS_REGIME = 62
    VOLATILE_REGIME = 63
    
    @classmethod
    def get_vocab_size(cls):
        return 64
    
    @classmethod
    def id_to_label(cls, idx: int) -> str:
        for name, value in vars(cls).items():
            if isinstance(value, int) and value == idx:
                return name
        return "UNKNOWN"

VOCAB = EventVocabulary()

# ============================================================================
# 2. HIERARCHICAL STRUCTURES
# ============================================================================

class EventScale(IntEnum):
    """Hierarchical scale levels"""
    MICRO = 1      # 1-5 steps
    MINI = 2       # 5-15 steps
    MESO = 3       # 15-50 steps
    MACRO = 4      # 50-150 steps
    GLOBAL = 5     # 150+ steps

@dataclass
class HierarchicalEvent:
    """Event node in hierarchy tree"""
    start: int
    end: int
    label: int
    label_name: str
    scale: EventScale
    event_type: str
    confidence: float
    metadata: Dict
    parent: Optional['HierarchicalEvent'] = None
    children: List['HierarchicalEvent'] = field(default_factory=list)
    
    @property
    def duration(self) -> int:
        return self.end - self.start + 1
    
    @property
    def depth(self) -> int:
        depth = 0
        node = self.parent
        while node is not None:
            depth += 1
            node = node.parent
        return depth
    
    def contains(self, other: 'HierarchicalEvent') -> bool:
        return (self.start <= other.start and 
                self.end >= other.end and
                self != other)
    
    def __repr__(self):
        indent = "  " * self.depth
        children_info = f" ({len(self.children)} children)" if self.children else ""
        return (f"{indent}[{self.start:03d}-{self.end:03d}] {self.label_name} "
                f"(scale={self.scale.name}){children_info}")

# ============================================================================
# 3. FIXED FEATURE EXTRACTOR
# ============================================================================

class MultiScaleFeatureExtractor:
    """Extract features at multiple temporal scales - FIXED VERSION"""
    
    def __init__(self, scales=[5, 10, 20, 50]):
        self.scales = scales
    
    def extract_features(self, x: torch.Tensor) -> Dict[str, torch.Tensor]:
        """
        x: [B, L] time series
        Returns: dict of features at multiple scales
        """
        B, L = x.shape
        device = x.device
        
        features = {}
        
        # First derivative
        dx = torch.diff(x, dim=1)  # [B, L-1]
        features['dx'] = F.pad(dx, (1, 0), value=0)  # [B, L]
        
        # Multi-scale rolling features
        for w in self.scales:
            if w >= L:
                continue
            
            x_3d = x.unsqueeze(1)  # [B, 1, L]
            kernel = torch.ones(1, 1, w, device=device) / w
            padding = w - 1
            
            # Rolling mean
            x_padded = F.pad(x_3d, (padding, 0), mode='replicate')
            rolling_mean = F.conv1d(x_padded, kernel).squeeze(1)
            features[f'mean_{w}'] = rolling_mean
            
            # Rolling std
            x_diff = x.unsqueeze(1) - rolling_mean.unsqueeze(1)
            x_diff_padded = F.pad(x_diff, (padding, 0), mode='replicate')
            rolling_var = F.conv1d(x_diff_padded ** 2, kernel).squeeze(1)
            features[f'std_{w}'] = torch.sqrt(rolling_var.clamp(min=1e-8))
            
            # Simple rolling slope
            slopes = self._compute_slopes(x, w)
            features[f'slope_{w}'] = slopes
        
        # Z-scores
        mean = x.mean(dim=1, keepdim=True)
        std = x.std(dim=1, keepdim=True).clamp(min=1e-8)
        features['zscore'] = (x - mean) / std
        
        return features
    
    def _compute_slopes(self, x: torch.Tensor, window: int) -> torch.Tensor:
        """Simple slope computation"""
        B, L = x.shape
        slopes = torch.zeros(B, L, device=x.device)
        
        for i in range(window, L):
            # Slope from start to end of window
            slopes[:, i] = (x[:, i] - x[:, i-window+1]) / window
        
        return slopes

# ============================================================================
# 4. STEP ENCODER
# ============================================================================

class StepWiseEncoder:
    """Encode each timestep - SIMPLIFIED"""
    
    def encode(self, x: torch.Tensor, features: Dict) -> torch.Tensor:
        B, L = x.shape
        device = x.device
        
        dx = features['dx']
        abs_dx = torch.abs(dx[:, 1:])
        
        if abs_dx.numel() == 0:
            return torch.full((B, L), VOCAB.FLAT, dtype=torch.long, device=device)
        
        # Quantiles
        q33, q66, q90 = torch.quantile(
            abs_dx.reshape(-1),
            torch.tensor([0.33, 0.66, 0.90], device=device)
        )
        
        epsilon = 0.1 * q33
        labels = torch.full((B, L), VOCAB.PAD, dtype=torch.long, device=device)
        
        for t in range(1, L):
            diff = dx[:, t]
            abs_diff = torch.abs(diff)
            
            # Classify
            flat_mask = abs_diff < epsilon
            labels[flat_mask, t] = VOCAB.FLAT
            
            up_mask = (diff > 0) & (~flat_mask)
            labels[up_mask & (abs_diff <= q33), t] = VOCAB.UP_SMALL
            labels[up_mask & (abs_diff > q33) & (abs_diff <= q66), t] = VOCAB.UP_MEDIUM
            labels[up_mask & (abs_diff > q66) & (abs_diff <= q90), t] = VOCAB.UP_LARGE
            labels[up_mask & (abs_diff > q90), t] = VOCAB.SPIKE_UP
            
            down_mask = (diff < 0) & (~flat_mask)
            labels[down_mask & (abs_diff <= q33), t] = VOCAB.DOWN_SMALL
            labels[down_mask & (abs_diff > q33) & (abs_diff <= q66), t] = VOCAB.DOWN_MEDIUM
            labels[down_mask & (abs_diff > q66) & (abs_diff <= q90), t] = VOCAB.DOWN_LARGE
            labels[down_mask & (abs_diff > q90), t] = VOCAB.SPIKE_DOWN
        
        return labels

# ============================================================================
# 5. SIMPLE DETECTORS
# ============================================================================

@dataclass
class SimpleSegment:
    start: int
    end: int
    label: int
    metadata: Dict = field(default_factory=dict)

class SimpleTrendDetector:
    """Simplified trend detector"""
    
    def detect(self, x: torch.Tensor, features: Dict, idx: int) -> List[SimpleSegment]:
        x_np = x.cpu().numpy()
        L = len(x_np)
        
        # Use slope if available
        if 'slope_20' in features:
            slopes = features['slope_20'][idx].cpu().numpy()
        else:
            slopes = np.gradient(x_np)
        
        # Find trend changes
        slope_sign = np.sign(slopes)
        slope_sign[np.abs(slopes) < 0.01] = 0
        
        changes = np.where(np.diff(slope_sign) != 0)[0] + 1
        breakpoints = np.concatenate([[0], changes, [L]])
        
        segments = []
        for i in range(len(breakpoints) - 1):
            start = breakpoints[i]
            end = breakpoints[i + 1] - 1
            
            if end - start < 5:
                continue
            
            avg_slope = slopes[start:end+1].mean()
            duration = end - start + 1
            
            # Classify
            if abs(avg_slope) < 0.01:
                label = VOCAB.FLAT_SEGMENT
            elif avg_slope > 0:
                if duration < 30:
                    label = VOCAB.UPTREND_SHORT
                elif duration < 80:
                    label = VOCAB.UPTREND_MEDIUM
                else:
                    label = VOCAB.UPTREND_LONG
            else:
                if duration < 30:
                    label = VOCAB.DOWNTREND_SHORT
                elif duration < 80:
                    label = VOCAB.DOWNTREND_MEDIUM
                else:
                    label = VOCAB.DOWNTREND_LONG
            
            segments.append(SimpleSegment(
                start=start,
                end=end,
                label=label,
                metadata={'slope': float(avg_slope)}
            ))
        
        return segments

class SimplePeakDetector:
    """Simplified peak detector"""
    
    def detect(self, x: torch.Tensor, idx: int) -> List[SimpleSegment]:
        x_np = x.cpu().numpy()
        events = []
        
        # Find peaks
        try:
            peaks, props = scipy_signal.find_peaks(x_np, prominence=0.2)
            for pk, prom in zip(peaks, props['prominences']):
                label = VOCAB.SHARP_PEAK if prom > 0.5 else VOCAB.LOCAL_PEAK
                events.append(SimpleSegment(
                    start=int(pk),
                    end=int(pk),
                    label=label,
                    metadata={'prominence': float(prom)}
                ))
        except:
            pass
        
        # Find troughs
        try:
            troughs, props = scipy_signal.find_peaks(-x_np, prominence=0.2)
            for tr, prom in zip(troughs, props['prominences']):
                label = VOCAB.SHARP_TROUGH if prom > 0.5 else VOCAB.LOCAL_TROUGH
                events.append(SimpleSegment(
                    start=int(tr),
                    end=int(tr),
                    label=label,
                    metadata={'prominence': float(prom)}
                ))
        except:
            pass
        
        return events

# ============================================================================
# 6. HIERARCHY BUILDER
# ============================================================================

class HierarchicalEventBuilder:
    """Build hierarchical structure"""
    
    def __init__(self):
        self.events = []
    
    def add_event(self, start: int, end: int, label: int, event_type: str,
                  confidence: float = 1.0, metadata: Optional[Dict] = None):
        duration = end - start + 1
        
        if duration <= 5:
            scale = EventScale.MICRO
        elif duration <= 15:
            scale = EventScale.MINI
        elif duration <= 50:
            scale = EventScale.MESO
        elif duration <= 150:
            scale = EventScale.MACRO
        else:
            scale = EventScale.GLOBAL
        
        event = HierarchicalEvent(
            start=start, end=end, label=label,
            label_name=VOCAB.id_to_label(label),
            scale=scale, event_type=event_type,
            confidence=confidence, metadata=metadata or {}
        )
        self.events.append(event)
    
    def build_hierarchy(self) -> List[HierarchicalEvent]:
        sorted_events = sorted(self.events, key=lambda e: (-e.scale, e.start))
        roots = []
        
        for event in sorted_events:
            parent = self._find_parent(event, sorted_events)
            if parent is None:
                roots.append(event)
            else:
                event.parent = parent
                parent.children.append(event)
        
        self._sort_children(roots)
        return roots
    
    def _find_parent(self, event: HierarchicalEvent,
                     all_events: List[HierarchicalEvent]) -> Optional[HierarchicalEvent]:
        candidates = [e for e in all_events if e != event and 
                     e.scale > event.scale and e.contains(event)]
        return min(candidates, key=lambda e: e.duration) if candidates else None
    
    def _sort_children(self, nodes: List[HierarchicalEvent]):
        for node in nodes:
            if node.children:
                node.children.sort(key=lambda c: c.start)
                self._sort_children(node.children)
    
    def get_flat_list(self, roots: List[HierarchicalEvent]) -> List[HierarchicalEvent]:
        result = []
        def traverse(node):
            result.append(node)
            for child in node.children:
                traverse(child)
        for root in roots:
            traverse(root)
        return result

# ============================================================================
# 7. HIERARCHICAL ANNOTATION
# ============================================================================

@dataclass
class HierarchicalAnnotation:
    sequence: torch.Tensor
    step_labels: torch.Tensor
    event_roots: List[HierarchicalEvent]
    all_events: List[HierarchicalEvent]
    
    def print_hierarchy(self, max_depth: int = 10):
        def print_tree(node: HierarchicalEvent, depth: int = 0):
            if depth > max_depth:
                return
            print(node)
            for child in node.children:
                print_tree(child, depth + 1)
        
        print(f"\nHierarchical Events (Total: {len(self.all_events)})")
        print("=" * 80)
        for root in self.event_roots:
            print_tree(root)
    
    def to_text(self) -> str:
        """Generate hierarchical text"""
        parts = []
        def traverse(node):
            depth_marker = ">" * node.depth
            parts.append(f"{depth_marker}[{node.start}-{node.end}]{node.label_name}")
            for child in node.children:
                traverse(child)
        for root in self.event_roots:
            traverse(root)
        return " ".join(parts)

# ============================================================================
# 8. DATASET
# ============================================================================

class HierarchicalEventDataset(Dataset):
    """Main dataset class - FIXED"""
    
    def __init__(self, x: torch.Tensor, verbose: bool = True):
        super().__init__()
        self.x = x
        B, L = x.shape
        
        if verbose:
            print(f"Processing {B} sequences of length {L}...")
        
        # Initialize
        self.feature_extractor = MultiScaleFeatureExtractor()
        self.step_encoder = StepWiseEncoder()
        self.trend_detector = SimpleTrendDetector()
        self.peak_detector = SimplePeakDetector()
        
        # Extract features
        if verbose:
            print("Extracting features...")
        self.features = self.feature_extractor.extract_features(x)
        
        # Encode steps
        if verbose:
            print("Encoding step labels...")
        self.step_labels = self.step_encoder.encode(x, self.features)
        
        # Build annotations
        if verbose:
            print("Building hierarchical structure...")
        
        self.annotations = []
        for i in range(B):
            if verbose and i % 50 == 0:
                print(f"  Sequence {i}/{B}...")
            
            builder = HierarchicalEventBuilder()
            
            # Detect trends
            trends = self.trend_detector.detect(x[i], self.features, i)
            for seg in trends:
                builder.add_event(seg.start, seg.end, seg.label, 'trend',
                                metadata=seg.metadata)
            
            # Detect peaks
            peaks = self.peak_detector.detect(x[i], i)
            for pk in peaks:
                builder.add_event(pk.start, pk.end, pk.label, 'peak',
                                metadata=pk.metadata)
            
            # Add global regime
            if 'slope_20' in self.features:
                avg_slope = self.features['slope_20'][i].mean().item()
            else:
                avg_slope = 0
            
            if avg_slope > 0.05:
                global_label = VOCAB.BULLISH_REGIME
            elif avg_slope < -0.05:
                global_label = VOCAB.BEARISH_REGIME
            else:
                global_label = VOCAB.SIDEWAYS_REGIME
            
            builder.add_event(0, L-1, global_label, 'regime')
            
            # Build hierarchy
            roots = builder.build_hierarchy()
            all_events = builder.get_flat_list(roots)
            
            self.annotations.append(HierarchicalAnnotation(
                sequence=x[i],
                step_labels=self.step_labels[i],
                event_roots=roots,
                all_events=all_events
            ))
        
        if verbose:
            print(f"✓ Complete! Avg events per sequence: {self._avg_events():.1f}")
    
    def _avg_events(self):
        return sum(len(a.all_events) for a in self.annotations) / len(self.annotations)
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        return self.annotations[idx]

# ============================================================================
# 9. TEST
# ============================================================================

if __name__ == "__main__":
    print("=" * 80)
    print("HIERARCHICAL EVENT LABELING SYSTEM - FIXED VERSION")
    print("=" * 80)
    
    # Generate data
    B, L = 20, 336
    print(f"\nGenerating {B} sequences of length {L}...")
    
    torch.manual_seed(42)
    t = torch.linspace(0, 4*np.pi, L)
    x = torch.zeros(B, L)
    
    for i in range(B):
        trend = 0.5 * torch.sin(t / 2) + 0.1 * t
        noise = 0.1 * torch.randn(L)
        spikes = torch.zeros(L)
        spike_idx = torch.randint(50, L-50, (2,))
        spikes[spike_idx] = torch.randn(2) * 2
        x[i] = trend + noise + spikes
    
    # Create dataset
    dataset = HierarchicalEventDataset(x, verbose=True)
    
    # Show example
    print("\n" + "=" * 80)
    print("EXAMPLE HIERARCHICAL ANNOTATION")
    print("=" * 80)
    
    ann = dataset[0]
    ann.print_hierarchy(max_depth=3)
    
    print("\n" + "=" * 80)
    print("TEXT OUTPUT")
    print("=" * 80)
    text = ann.to_text()
    print(text[:500] + "...")
    print(f"\nTotal tokens: ~{len(text.split())}")
    
    print("\n" + "=" * 80)
    print("✓ System working correctly!")
    print("=" * 80)

HIERARCHICAL EVENT LABELING SYSTEM - FIXED VERSION

Generating 20 sequences of length 336...
Processing 20 sequences of length 336...
Extracting features...
Encoding step labels...
Building hierarchical structure...
  Sequence 0/20...
✓ Complete! Avg events per sequence: 89.0

EXAMPLE HIERARCHICAL ANNOTATION

Hierarchical Events (Total: 85)
[000-335] SIDEWAYS_REGIME (scale=GLOBAL) (61 children)
  [000-020] FLAT_SEGMENT (scale=MESO) (3 children)
    [003-003] LOCAL_TROUGH (scale=MICRO)
    [009-009] LOCAL_PEAK (scale=MICRO)
    [011-011] LOCAL_TROUGH (scale=MICRO)
  [021-027] UPTREND_SHORT (scale=MINI) (1 children)
    [027-027] LOCAL_PEAK (scale=MICRO)
  [032-032] LOCAL_TROUGH (scale=MICRO)
  [035-035] LOCAL_PEAK (scale=MICRO)
  [040-040] LOCAL_TROUGH (scale=MICRO)
  [045-045] LOCAL_PEAK (scale=MICRO)
  [046-046] LOCAL_TROUGH (scale=MICRO)
  [053-053] LOCAL_PEAK (scale=MICRO)
  [056-056] LOCAL_TROUGH (scale=MICRO)
  [061-061] LOCAL_PEAK (scale=MICRO)
  [062-062] LOCAL_TROUGH (scale=MIC

In [33]:
"""
HIERARCHICAL TIME SERIES EVENT LABELING SYSTEM
===============================================

A comprehensive system for detecting and labeling events in time series data
with hierarchical structure preservation.

Workflow:
    1. Define vocabulary and hierarchical structures
    2. Extract multi-scale features from raw time series
    3. Encode step-wise labels for each timestep
    4. Detect higher-level events (trends, peaks, volatility, change points)
    5. Build hierarchical event tree
    6. Generate training text in various formats

Author: Sachith
Date: December 2024
"""

import torch
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset
from dataclasses import dataclass, field
from typing import List, Dict, Optional
from scipy import signal as scipy_signal
from enum import IntEnum
import warnings
warnings.filterwarnings('ignore')


# ============================================================================
# SECTION 1: CORE DATA STRUCTURES
# ============================================================================

class EventScale(IntEnum):
    """Hierarchical scale levels for events"""
    MICRO = 1      # 1-5 timesteps (spikes, single points)
    MINI = 2       # 5-15 timesteps (very short segments)
    MESO = 3       # 15-50 timesteps (medium segments, local patterns)
    MACRO = 4      # 50-150 timesteps (major trends)
    GLOBAL = 5     # 150+ timesteps (full sequence characteristics)


@dataclass
class EventVocabulary:
    """
    Complete event vocabulary with 64 distinct labels.
    
    Categories:
        - Special tokens (0-2)
        - Step movements (3-10)
        - Trend segments (20-26)
        - Peaks/troughs (30-33)
        - Volatility regimes (40-43)
        - Change points (50-51)
        - Global regimes (60-63)
    """
    # Special tokens
    PAD = 0
    MASK = 1
    FLAT = 2
    
    # Step-level movements
    UP_SMALL = 3
    UP_MEDIUM = 4
    UP_LARGE = 5
    DOWN_SMALL = 6
    DOWN_MEDIUM = 7
    DOWN_LARGE = 8
    SPIKE_UP = 9
    SPIKE_DOWN = 10
    
    # Trend segments
    UPTREND_SHORT = 20
    UPTREND_MEDIUM = 21
    UPTREND_LONG = 22
    DOWNTREND_SHORT = 23
    DOWNTREND_MEDIUM = 24
    DOWNTREND_LONG = 25
    FLAT_SEGMENT = 26
    
    # Peaks and troughs
    LOCAL_PEAK = 30
    SHARP_PEAK = 31
    LOCAL_TROUGH = 32
    SHARP_TROUGH = 33
    
    # Volatility regimes
    LOW_VOLATILITY = 40
    NORMAL_VOLATILITY = 41
    HIGH_VOLATILITY = 42
    VOLATILITY_SPIKE = 43
    
    # Change points
    MEAN_SHIFT_UP = 50
    MEAN_SHIFT_DOWN = 51
    
    # Global regimes
    BULLISH_REGIME = 60
    BEARISH_REGIME = 61
    SIDEWAYS_REGIME = 62
    VOLATILE_REGIME = 63
    
    @classmethod
    def get_vocab_size(cls) -> int:
        """Return total vocabulary size"""
        return 64
    
    @classmethod
    def id_to_label(cls, idx: int) -> str:
        """Convert label ID to string name"""
        for name, value in vars(cls).items():
            if isinstance(value, int) and value == idx:
                return name
        return "UNKNOWN"


@dataclass
class HierarchicalEvent:
    """
    Event node in hierarchical tree structure.
    
    Attributes:
        start: Starting timestep index
        end: Ending timestep index
        label: Vocabulary ID
        label_name: Human-readable label
        scale: Hierarchical scale level
        event_type: Category (trend/peak/volatility/changepoint/regime)
        confidence: Detection confidence score
        metadata: Additional event-specific information
        parent: Parent event in hierarchy (None for root)
        children: List of child events
    """
    start: int
    end: int
    label: int
    label_name: str
    scale: EventScale
    event_type: str
    confidence: float
    metadata: Dict
    parent: Optional['HierarchicalEvent'] = None
    children: List['HierarchicalEvent'] = field(default_factory=list)
    
    @property
    def duration(self) -> int:
        """Duration in timesteps"""
        return self.end - self.start + 1
    
    @property
    def depth(self) -> int:
        """Depth in hierarchy tree (0 = root)"""
        depth = 0
        node = self.parent
        while node is not None:
            depth += 1
            node = node.parent
        return depth
    
    def contains(self, other: 'HierarchicalEvent') -> bool:
        """Check if this event fully contains another event"""
        return (self.start <= other.start and 
                self.end >= other.end and
                self != other)
    
    def __repr__(self):
        indent = "  " * self.depth
        children_info = f" ({len(self.children)} children)" if self.children else ""
        return (f"{indent}[{self.start:03d}-{self.end:03d}] {self.label_name} "
                f"(scale={self.scale.name}){children_info}")


# Global vocabulary instance
VOCAB = EventVocabulary()


# ============================================================================
# SECTION 2: FEATURE EXTRACTION
# ============================================================================

class MultiScaleFeatureExtractor:
    """
    Extract features at multiple temporal scales using efficient convolutions.
    
    Features computed:
        - First derivative (dx)
        - Rolling mean at multiple window sizes
        - Rolling standard deviation (volatility)
        - Rolling slope (trend strength)
        - Z-scores for anomaly detection
    
    Args:
        scales: List of window sizes for rolling features
    """
    
    def __init__(self, scales: List[int] = [5, 10, 20, 50]):
        self.scales = scales
    
    def extract_features(self, x: torch.Tensor) -> Dict[str, torch.Tensor]:
        """
        Extract multi-scale features from time series batch.
        
        Args:
            x: Time series tensor of shape [B, L]
        
        Returns:
            Dictionary of feature tensors, each shape [B, L]:
                - 'dx': First derivative
                - 'mean_{w}': Rolling mean with window w
                - 'std_{w}': Rolling std with window w
                - 'slope_{w}': Rolling slope with window w
                - 'zscore': Normalized z-scores
        """
        B, L = x.shape
        device = x.device
        features = {}
        
        # First derivative (rate of change)
        dx = torch.diff(x, dim=1)  # [B, L-1]
        features['dx'] = F.pad(dx, (1, 0), value=0)  # [B, L]
        
        # Multi-scale rolling features
        for w in self.scales:
            if w >= L:
                continue
            
            x_3d = x.unsqueeze(1)  # [B, 1, L]
            kernel = torch.ones(1, 1, w, device=device) / w
            padding = w - 1
            
            # Rolling mean using efficient convolution
            x_padded = F.pad(x_3d, (padding, 0), mode='replicate')
            rolling_mean = F.conv1d(x_padded, kernel).squeeze(1)
            features[f'mean_{w}'] = rolling_mean
            
            # Rolling standard deviation (volatility measure)
            x_diff = x.unsqueeze(1) - rolling_mean.unsqueeze(1)
            x_diff_padded = F.pad(x_diff, (padding, 0), mode='replicate')
            rolling_var = F.conv1d(x_diff_padded ** 2, kernel).squeeze(1)
            features[f'std_{w}'] = torch.sqrt(rolling_var.clamp(min=1e-8))
            
            # Rolling slope (trend direction and strength)
            slopes = self._compute_slopes(x, w)
            features[f'slope_{w}'] = slopes
        
        # Z-scores for outlier detection
        mean = x.mean(dim=1, keepdim=True)
        std = x.std(dim=1, keepdim=True).clamp(min=1e-8)
        features['zscore'] = (x - mean) / std
        
        return features
    
    def _compute_slopes(self, x: torch.Tensor, window: int) -> torch.Tensor:
        """
        Compute rolling linear slope over window.
        
        Simple implementation: slope = (end - start) / window
        """
        B, L = x.shape
        slopes = torch.zeros(B, L, device=x.device)
        
        for i in range(window, L):
            slopes[:, i] = (x[:, i] - x[:, i-window+1]) / window
        
        return slopes


# ============================================================================
# SECTION 3: STEP-WISE LABEL ENCODING
# ============================================================================

class StepWiseEncoder:
    """
    Encode each timestep with symbolic movement labels.
    
    Labels based on magnitude of first derivative:
        - FLAT: negligible change
        - UP/DOWN_SMALL/MEDIUM/LARGE: quantile-based magnitude bins
        - SPIKE_UP/DOWN: extreme changes (>90th percentile)
    """
    
    def encode(self, x: torch.Tensor, features: Dict) -> torch.Tensor:
        """
        Encode step-wise movement labels for entire batch.
        
        Args:
            x: Time series tensor [B, L]
            features: Feature dictionary from MultiScaleFeatureExtractor
        
        Returns:
            Label tensor [B, L] with vocabulary IDs
        """
        B, L = x.shape
        device = x.device
        
        dx = features['dx']
        abs_dx = torch.abs(dx[:, 1:])  # Skip first padded value
        
        if abs_dx.numel() == 0:
            return torch.full((B, L), VOCAB.FLAT, dtype=torch.long, device=device)
        
        # Compute quantiles for adaptive thresholding
        q33, q66, q90 = torch.quantile(
            abs_dx.reshape(-1),
            torch.tensor([0.33, 0.66, 0.90], device=device)
        )
        
        epsilon = 0.1 * q33  # Flat threshold
        labels = torch.full((B, L), VOCAB.PAD, dtype=torch.long, device=device)
        
        # Classify each timestep
        for t in range(1, L):
            diff = dx[:, t]
            abs_diff = torch.abs(diff)
            
            # Flat (negligible change)
            flat_mask = abs_diff < epsilon
            labels[flat_mask, t] = VOCAB.FLAT
            
            # Upward movements
            up_mask = (diff > 0) & (~flat_mask)
            labels[up_mask & (abs_diff <= q33), t] = VOCAB.UP_SMALL
            labels[up_mask & (abs_diff > q33) & (abs_diff <= q66), t] = VOCAB.UP_MEDIUM
            labels[up_mask & (abs_diff > q66) & (abs_diff <= q90), t] = VOCAB.UP_LARGE
            labels[up_mask & (abs_diff > q90), t] = VOCAB.SPIKE_UP
            
            # Downward movements
            down_mask = (diff < 0) & (~flat_mask)
            labels[down_mask & (abs_diff <= q33), t] = VOCAB.DOWN_SMALL
            labels[down_mask & (abs_diff > q33) & (abs_diff <= q66), t] = VOCAB.DOWN_MEDIUM
            labels[down_mask & (abs_diff > q66) & (abs_diff <= q90), t] = VOCAB.DOWN_LARGE
            labels[down_mask & (abs_diff > q90), t] = VOCAB.SPIKE_DOWN
        
        return labels


# ============================================================================
# SECTION 4: EVENT DETECTORS
# ============================================================================

@dataclass
class SimpleSegment:
    """Simple segment representation for detector outputs"""
    start: int
    end: int
    label: int
    metadata: Dict = field(default_factory=dict)


class TrendSegmentDetector:
    """
    Detect trend segments using slope sign changes.
    
    Algorithm:
        1. Compute rolling slopes
        2. Find sign changes (trend reversals)
        3. Classify segments by direction and duration
    """
    
    def detect(self, x: torch.Tensor, features: Dict, idx: int) -> List[SimpleSegment]:
        """
        Detect trend segments in single sequence.
        
        Args:
            x: Single sequence tensor [L]
            features: Feature dictionary
            idx: Sequence index in batch
        
        Returns:
            List of trend segments
        """
        x_np = x.cpu().numpy()
        L = len(x_np)
        
        # Get slopes
        if 'slope_20' in features:
            slopes = features['slope_20'][idx].cpu().numpy()
        else:
            slopes = np.gradient(x_np)
        
        # Find trend direction changes
        slope_sign = np.sign(slopes)
        slope_sign[np.abs(slopes) < 0.01] = 0  # Flat threshold
        
        changes = np.where(np.diff(slope_sign) != 0)[0] + 1
        breakpoints = np.concatenate([[0], changes, [L]])
        
        segments = []
        for i in range(len(breakpoints) - 1):
            start = breakpoints[i]
            end = breakpoints[i + 1] - 1
            
            if end - start < 5:  # Skip very short segments
                continue
            
            avg_slope = slopes[start:end+1].mean()
            duration = end - start + 1
            
            # Classify segment
            if abs(avg_slope) < 0.01:
                label = VOCAB.FLAT_SEGMENT
            elif avg_slope > 0:
                if duration < 30:
                    label = VOCAB.UPTREND_SHORT
                elif duration < 80:
                    label = VOCAB.UPTREND_MEDIUM
                else:
                    label = VOCAB.UPTREND_LONG
            else:
                if duration < 30:
                    label = VOCAB.DOWNTREND_SHORT
                elif duration < 80:
                    label = VOCAB.DOWNTREND_MEDIUM
                else:
                    label = VOCAB.DOWNTREND_LONG
            
            segments.append(SimpleSegment(
                start=start,
                end=end,
                label=label,
                metadata={'slope': float(avg_slope)}
            ))
        
        return segments


class PeakTroughDetector:
    """
    Detect peaks and troughs using scipy's find_peaks.
    
    Classifies peaks/troughs by:
        - Prominence: How much the peak stands out
        - Type: Sharp vs broad based on width
    """
    
    def detect(self, x: torch.Tensor, idx: int) -> List[SimpleSegment]:
        """
        Detect peaks and troughs in single sequence.
        
        Args:
            x: Single sequence tensor [L]
            idx: Sequence index (unused but kept for consistency)
        
        Returns:
            List of peak/trough events
        """
        x_np = x.cpu().numpy()
        events = []
        
        # Find peaks
        try:
            peaks, props = scipy_signal.find_peaks(x_np, prominence=0.2)
            for pk, prom in zip(peaks, props['prominences']):
                label = VOCAB.SHARP_PEAK if prom > 0.5 else VOCAB.LOCAL_PEAK
                events.append(SimpleSegment(
                    start=int(pk),
                    end=int(pk),
                    label=label,
                    metadata={'prominence': float(prom)}
                ))
        except:
            pass
        
        # Find troughs (peaks of inverted signal)
        try:
            troughs, props = scipy_signal.find_peaks(-x_np, prominence=0.2)
            for tr, prom in zip(troughs, props['prominences']):
                label = VOCAB.SHARP_TROUGH if prom > 0.5 else VOCAB.LOCAL_TROUGH
                events.append(SimpleSegment(
                    start=int(tr),
                    end=int(tr),
                    label=label,
                    metadata={'prominence': float(prom)}
                ))
        except:
            pass
        
        return events


class VolatilityRegimeDetector:
    """
    Detect volatility regimes using rolling standard deviation.
    
    Classifies regimes by quantile thresholds:
        - LOW: Below 25th percentile
        - NORMAL: 25th-75th percentile
        - HIGH: Above 75th percentile
        - SPIKE: Above 90th percentile
    """
    
    def detect(self, x: torch.Tensor, features: Dict, idx: int) -> List[SimpleSegment]:
        """
        Detect volatility regimes in single sequence.
        
        Args:
            x: Single sequence tensor [L]
            features: Feature dictionary
            idx: Sequence index in batch
        
        Returns:
            List of volatility regime segments
        """
        if 'std_20' not in features:
            return []
        
        vol = features['std_20'][idx].cpu().numpy()
        L = len(vol)
        
        # Compute quantile thresholds
        q25, q75, q90 = np.percentile(vol, [25, 75, 90])
        
        # Classify each point
        vol_levels = np.zeros(L, dtype=int)
        vol_levels[vol <= q25] = 0  # low
        vol_levels[(vol > q25) & (vol <= q75)] = 1  # normal
        vol_levels[(vol > q75) & (vol <= q90)] = 2  # high
        vol_levels[vol > q90] = 3  # spike
        
        # Find regime changes
        changes = np.where(np.diff(vol_levels) != 0)[0] + 1
        breakpoints = np.concatenate([[0], changes, [L]])
        
        regimes = []
        for i in range(len(breakpoints) - 1):
            start = breakpoints[i]
            end = breakpoints[i + 1] - 1
            
            if end - start < 5:  # Skip very short regimes
                continue
            
            level_code = vol_levels[start]
            avg_vol = vol[start:end+1].mean()
            
            # Map to vocabulary
            label_map = {
                0: VOCAB.LOW_VOLATILITY,
                1: VOCAB.NORMAL_VOLATILITY,
                2: VOCAB.HIGH_VOLATILITY,
                3: VOCAB.VOLATILITY_SPIKE
            }
            
            regimes.append(SimpleSegment(
                start=start,
                end=end,
                label=label_map[level_code],
                metadata={'avg_volatility': float(avg_vol)}
            ))
        
        return regimes


# ============================================================================
# SECTION 5: HIERARCHICAL STRUCTURE BUILDER
# ============================================================================

class HierarchicalEventBuilder:
    """
    Build hierarchical event tree from flat event list.
    
    Process:
        1. Classify each event's scale based on duration
        2. Sort events by scale (largest first)
        3. Build parent-child relationships via containment
        4. Sort children by temporal order
    """
    
    def __init__(self):
        self.events: List[HierarchicalEvent] = []
    
    def add_event(self, start: int, end: int, label: int, event_type: str,
                  confidence: float = 1.0, metadata: Optional[Dict] = None):
        """
        Add event to collection with automatic scale classification.
        
        Args:
            start: Starting timestep
            end: Ending timestep
            label: Vocabulary ID
            event_type: Category string
            confidence: Detection confidence score
            metadata: Additional information dictionary
        """
        duration = end - start + 1
        
        # Determine hierarchical scale
        if duration <= 5:
            scale = EventScale.MICRO
        elif duration <= 15:
            scale = EventScale.MINI
        elif duration <= 50:
            scale = EventScale.MESO
        elif duration <= 150:
            scale = EventScale.MACRO
        else:
            scale = EventScale.GLOBAL
        
        event = HierarchicalEvent(
            start=start, end=end, label=label,
            label_name=VOCAB.id_to_label(label),
            scale=scale, event_type=event_type,
            confidence=confidence, metadata=metadata or {}
        )
        self.events.append(event)
    
    def build_hierarchy(self) -> List[HierarchicalEvent]:
        """
        Build hierarchical tree structure.
        
        Returns:
            List of root events (events with no parent)
        """
        # Sort by scale (largest first), then by start position
        sorted_events = sorted(self.events, key=lambda e: (-e.scale, e.start))
        roots = []
        
        # Build parent-child relationships
        for event in sorted_events:
            parent = self._find_parent(event, sorted_events)
            if parent is None:
                roots.append(event)
            else:
                event.parent = parent
                parent.children.append(event)
        
        # Sort children within each parent
        self._sort_children(roots)
        return roots
    
    def _find_parent(self, event: HierarchicalEvent,
                     all_events: List[HierarchicalEvent]) -> Optional[HierarchicalEvent]:
        """Find smallest event that contains this event (most specific parent)"""
        candidates = [e for e in all_events if e != event and 
                     e.scale > event.scale and e.contains(event)]
        return min(candidates, key=lambda e: e.duration) if candidates else None
    
    def _sort_children(self, nodes: List[HierarchicalEvent]):
        """Recursively sort children by start position"""
        for node in nodes:
            if node.children:
                node.children.sort(key=lambda c: c.start)
                self._sort_children(node.children)
    
    def get_flat_list(self, roots: List[HierarchicalEvent]) -> List[HierarchicalEvent]:
        """Get flattened list via depth-first traversal"""
        result = []
        def traverse(node):
            result.append(node)
            for child in node.children:
                traverse(child)
        for root in roots:
            traverse(root)
        return result


# ============================================================================
# SECTION 6: HIERARCHICAL ANNOTATION
# ============================================================================

@dataclass
class HierarchicalAnnotation:
    """
    Complete hierarchical annotation for one sequence.
    
    Attributes:
        sequence: Original time series [L]
        step_labels: Step-wise labels [L]
        event_roots: Root nodes of hierarchy tree
        all_events: Flattened list of all events
    """
    sequence: torch.Tensor
    step_labels: torch.Tensor
    event_roots: List[HierarchicalEvent]
    all_events: List[HierarchicalEvent]
    
    def print_hierarchy(self, max_depth: int = 10):
        """Pretty print hierarchical structure"""
        def print_tree(node: HierarchicalEvent, depth: int = 0):
            if depth > max_depth:
                return
            print(node)
            for child in node.children:
                print_tree(child, depth + 1)
        
        print(f"\nHierarchical Events (Total: {len(self.all_events)})")
        print("=" * 80)
        for root in self.event_roots:
            print_tree(root)
    
    def get_events_at_scale(self, scale: EventScale) -> List[HierarchicalEvent]:
        """Get all events at specific hierarchical scale"""
        return [e for e in self.all_events if e.scale == scale]
    
    def get_events_in_range(self, start: int, end: int) -> List[HierarchicalEvent]:
        """Get all events overlapping with time range"""
        return [e for e in self.all_events 
                if not (e.end < start or e.start > end)]
    
    def to_text(self, format: str = 'depth_marked') -> str:
        """
        Generate text representation.
        
        Args:
            format: Output format
                - 'depth_marked': Depth indicators with events
                - 'flat': Simple sequential list
                - 'narrative': Natural language description
        
        Returns:
            Text string for language model training
        """
        if format == 'depth_marked':
            return self._depth_marked_text()
        elif format == 'flat':
            return self._flat_text()
        elif format == 'narrative':
            return self._narrative_text()
        else:
            raise ValueError(f"Unknown format: {format}")
    
    def _depth_marked_text(self) -> str:
        """Depth markers indicate nesting: > >> >>> etc."""
        parts = []
        def traverse(node):
            depth_marker = ">" * node.depth
            parts.append(f"{depth_marker}[{node.start}-{node.end}]{node.label_name}")
            for child in node.children:
                traverse(child)
        for root in self.event_roots:
            traverse(root)
        return " ".join(parts)
    
    def _flat_text(self) -> str:
        """Simple sequential list (loses hierarchy)"""
        events = sorted(self.all_events, key=lambda e: e.start)
        return " ".join(f"[{e.start}-{e.end}]{e.label_name}" for e in events)
    
    def _narrative_text(self) -> str:
        """Natural language with hierarchical context"""
        sentences = []
        
        # Start with global view
        global_events = self.get_events_at_scale(EventScale.GLOBAL)
        if global_events:
            sentences.append(
                f"Overall: {global_events[0].label_name.lower().replace('_', ' ')}."
            )
        
        # Describe macro events
        macro_events = self.get_events_at_scale(EventScale.MACRO)
        if macro_events:
            sentences.append(f"{len(macro_events)} major segments detected.")
            for event in macro_events[:3]:
                desc = event.label_name.lower().replace('_', ' ')
                sentences.append(f"[{event.start}-{event.end}]: {desc}")
                if event.children:
                    nested = ", ".join(set(c.event_type for c in event.children))
                    sentences.append(f"  (contains: {nested})")
        
        return " ".join(sentences)


# ============================================================================
# SECTION 7: MAIN DATASET CLASS
# ============================================================================

class HierarchicalEventDataset(Dataset):
    """
    Main dataset class for hierarchical event labeling.
    
    Processing pipeline:
        1. Extract multi-scale features
        2. Encode step-wise labels
        3. Detect events (trends, peaks, volatility)
        4. Add global regime classification
        5. Build hierarchical structure
        6. Create annotations
    
    Args:
        x: Time series tensor [B, L]
        verbose: Print progress messages
    """
    
    def __init__(self, x: torch.Tensor, verbose: bool = True):
        super().__init__()
        
        if x.dim() != 2:
            raise ValueError("Expected x with shape [B, L]")
        
        self.x = x
        B, L = x.shape
        
        if verbose:
            print(f"\n{'='*80}")
            print(f"INITIALIZING HIERARCHICAL EVENT DATASET")
            print(f"{'='*80}")
            print(f"Sequences: {B}")
            print(f"Length: {L}")
        
        # Initialize components
        self.feature_extractor = MultiScaleFeatureExtractor()
        self.step_encoder = StepWiseEncoder()
        self.trend_detector = TrendSegmentDetector()
        self.peak_detector = PeakTroughDetector()
        self.vol_detector = VolatilityRegimeDetector()
        
        # STEP 1: Extract features
        if verbose:
            print(f"\n[1/4] Extracting multi-scale features...")
        self.features = self.feature_extractor.extract_features(x)
        if verbose:
            print(f"      ✓ Computed {len(self.features)} feature types")
        
        # STEP 2: Encode step labels
        if verbose:
            print(f"[2/4] Encoding step-wise labels...")
        self.step_labels = self.step_encoder.encode(x, self.features)
        if verbose:
            print(f"      ✓ Encoded {B * L} timesteps")
        
        # STEP 3: Detect events and build hierarchy
        if verbose:
            print(f"[3/4] Detecting events and building hierarchy...")
        
        self.annotations = []
        for i in range(B):
            if verbose and i % 50 == 0:
                print(f"      Processing sequence {i}/{B}...")
            
            annotation = self._build_annotation(i, L)
            self.annotations.append(annotation)
        
        # STEP 4: Compute statistics
        if verbose:
            print(f"[4/4] Computing statistics...")
            self._print_statistics()
            print(f"\n{'='*80}")
            print(f"✓ DATASET READY")
            print(f"{'='*80}\n")
    
    def _build_annotation(self, idx: int, L: int) -> HierarchicalAnnotation:
        """Build complete hierarchical annotation for one sequence"""
        
        builder = HierarchicalEventBuilder()
        
        # Detect all event types
        trends = self.trend_detector.detect(self.x[idx], self.features, idx)
        peaks = self.peak_detector.detect(self.x[idx], idx)
        vol_regimes = self.vol_detector.detect(self.x[idx], self.features, idx)
        
        # Add trend segments
        for seg in trends:
            builder.add_event(seg.start, seg.end, seg.label, 'trend',
                            confidence=0.9, metadata=seg.metadata)
        
        # Add peaks/troughs
        for pk in peaks:
            builder.add_event(pk.start, pk.end, pk.label, 'peak',
                            confidence=0.85, metadata=pk.metadata)
        
        # Add volatility regimes
        for vr in vol_regimes:
            builder.add_event(vr.start, vr.end, vr.label, 'volatility',
                            confidence=0.8, metadata=vr.metadata)
        
        # Add global regime
        global_label = self._classify_global_regime(idx)
        builder.add_event(0, L-1, global_label, 'regime', confidence=0.7)
        
        # Build hierarchy
        roots = builder.build_hierarchy()
        all_events = builder.get_flat_list(roots)
        
        return HierarchicalAnnotation(
            sequence=self.x[idx],
            step_labels=self.step_labels[idx],
            event_roots=roots,
            all_events=all_events
        )
    
    def _classify_global_regime(self, idx: int) -> int:
        """Classify overall sequence regime"""
        if 'slope_20' in self.features:
            avg_slope = self.features['slope_20'][idx].mean().item()
        else:
            avg_slope = 0
        
        if avg_slope > 0.05:
            return VOCAB.BULLISH_REGIME
        elif avg_slope < -0.05:
            return VOCAB.BEARISH_REGIME
        else:
            return VOCAB.SIDEWAYS_REGIME
    
    def _print_statistics(self):
        """Print dataset statistics"""
        total_events = sum(len(a.all_events) for a in self.annotations)
        avg_events = total_events / len(self.annotations)
        
        # Count by scale
        scale_counts = {scale: 0 for scale in EventScale}
        for ann in self.annotations:
            for event in ann.all_events:
                scale_counts[event.scale] += 1
        
        print(f"      Total events: {total_events}")
        print(f"      Avg per sequence: {avg_events:.1f}")
        print(f"      By scale:")
        for scale in EventScale:
            count = scale_counts[scale]
            avg = count / len(self.annotations)
            print(f"        {scale.name:.<12} {avg:>6.1f} per sequence")
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        return self.annotations[idx]
    
    def get_statistics(self) -> Dict:
        """Get comprehensive statistics"""
        total_events = sum(len(a.all_events) for a in self.annotations)
        
        return {
            'num_sequences': len(self.annotations),
            'sequence_length': len(self.annotations[0].sequence),
            'vocab_size': VOCAB.get_vocab_size(),
            'total_events': total_events,
            'avg_events_per_sequence': total_events / len(self.annotations),
        }


# ============================================================================
# SECTION 8: TEXT GENERATION FOR LM TRAINING
# ============================================================================

class TextCorpusGenerator:
    """
    Generate training text in various formats.
    
    Formats:
        - depth_marked: Hierarchical with depth indicators (>)
        - flat: Simple sequential list
        - narrative: Natural language description
    """
    
    @staticmethod
    def generate_corpus(dataset: HierarchicalEventDataset, 
                       format: str = 'depth_marked') -> List[str]:
        """
        Generate text corpus for all sequences.
        
        Args:
            dataset: HierarchicalEventDataset instance
            format: Text format
        
        Returns:
            List of text strings, one per sequence
        """
        corpus = []
        for annotation in dataset.annotations:
            text = annotation.to_text(format=format)
            corpus.append(text)
        return corpus
    
    @staticmethod
    def estimate_tokens(corpus: List[str]) -> Dict:
        """Estimate token counts for corpus"""
        total_tokens = sum(len(text.split()) for text in corpus)
        total_chars = sum(len(text) for text in corpus)
        
        return {
            'num_documents': len(corpus),
            'total_tokens': total_tokens,
            'total_chars': total_chars,
            'avg_tokens_per_doc': total_tokens / len(corpus),
            'avg_chars_per_doc': total_chars / len(corpus),
        }


# ============================================================================
# SECTION 9: DEMONSTRATION & TESTING
# ============================================================================

def generate_synthetic_data(B: int = 50, L: int = 336, seed: int = 42) -> torch.Tensor:
    """
    Generate realistic synthetic time series.
    
    Components:
        - Multi-scale sinusoidal trends
        - Volatility clusters
        - Random spikes
        - Local corrections (creates nested events)
    
    Args:
        B: Batch size (number of sequences)
        L: Sequence length
        seed: Random seed
    
    Returns:
        Tensor of shape [B, L]
    """
    torch.manual_seed(seed)
    np.random.seed(seed)
    
    t = torch.linspace(0, 4*np.pi, L)
    x = torch.zeros(B, L)
    
    for i in range(B):
        # Base trend (multiple scales)
        trend = 0.5 * torch.sin(t / 2) + 0.1 * t
        
        # Add volatility clusters
        vol_modulator = 0.1 + 0.2 * (torch.sin(3 * t) > 0).float()
        noise = torch.randn(L) * vol_modulator
        
        # Add random spikes
        num_spikes = np.random.randint(2, 5)
        spike_indices = torch.randint(50, L-50, (num_spikes,))
        spikes = torch.zeros(L)
        spikes[spike_indices] = torch.randn(num_spikes) * 2
        
        # Add local correction (creates nested structure)
        if i % 3 == 0:
            # Dip in middle of uptrend
            start = L // 2
            end = start + 30
            x[i, start:end] = x[i, start:end] - 0.5
        
        x[i] = trend + noise + spikes
    
    return x


def demonstrate_system():
    """Run complete demonstration of the system"""
    
    print("\n" + "="*80)
    print("HIERARCHICAL TIME SERIES EVENT LABELING SYSTEM")
    print("Demonstration")
    print("="*80)
    
    # Generate data
    B, L = 20, 336
    print(f"\nGenerating {B} synthetic sequences of length {L}...")
    x = generate_synthetic_data(B, L)
    
    # Create dataset
    dataset = HierarchicalEventDataset(x, verbose=True)
    
    # Show example annotation
    print("\n" + "="*80)
    print("EXAMPLE: HIERARCHICAL STRUCTURE")
    print("="*80)
    
    ann = dataset[0]
    ann.print_hierarchy(max_depth=3)
    
    # Show events by scale
    print("\n" + "="*80)
    print("EVENTS BY HIERARCHICAL SCALE")
    print("="*80)
    
    for scale in EventScale:
        events = ann.get_events_at_scale(scale)
        print(f"\n{scale.name} ({len(events)} events):")
        for e in events[:5]:
            print(f"  [{e.start:03d}-{e.end:03d}] {e.label_name}")
    
    # Generate text in different formats
    print("\n" + "="*80)
    print("TEXT GENERATION FOR LM TRAINING")
    print("="*80)
    
    formats = ['depth_marked', 'flat', 'narrative']
    for fmt in formats:
        text = ann.to_text(format=fmt)
        tokens = len(text.split())
        chars = len(text)
        print(f"\n{fmt.upper()}:")
        print(f"  Tokens: {tokens}, Chars: {chars}")
        print(f"  Preview: {text[:200]}...")
    
    # Generate full corpus
    print("\n" + "="*80)
    print("FULL CORPUS STATISTICS")
    print("="*80)
    
    text_gen = TextCorpusGenerator()
    corpus = text_gen.generate_corpus(dataset, format='depth_marked')
    stats = text_gen.estimate_tokens(corpus)
    
    for key, value in stats.items():
        print(f"  {key}: {value:,.1f}" if isinstance(value, float) else f"  {key}: {value:,}")
    
    print("\n" + "="*80)
    print("✓ DEMONSTRATION COMPLETE")
    print("="*80)
    print("\nThe system is ready for processing real time series data.")
    print("Simply create a dataset with: dataset = HierarchicalEventDataset(your_data)")


# ============================================================================
# MAIN ENTRY POINT
# ============================================================================

if __name__ == "__main__":
    demonstrate_system()


HIERARCHICAL TIME SERIES EVENT LABELING SYSTEM
Demonstration

Generating 20 synthetic sequences of length 336...

INITIALIZING HIERARCHICAL EVENT DATASET
Sequences: 20
Length: 336

[1/4] Extracting multi-scale features...
      ✓ Computed 14 feature types
[2/4] Encoding step-wise labels...
      ✓ Encoded 6720 timesteps
[3/4] Detecting events and building hierarchy...
      Processing sequence 0/20...
[4/4] Computing statistics...
      Total events: 3039
      Avg per sequence: 151.9
      By scale:
        MICRO.......  125.2 per sequence
        MINI........   17.6 per sequence
        MESO........    8.1 per sequence
        MACRO.......    0.0 per sequence
        GLOBAL......    1.0 per sequence

✓ DATASET READY


EXAMPLE: HIERARCHICAL STRUCTURE

Hierarchical Events (Total: 153)
[000-335] SIDEWAYS_REGIME (scale=GLOBAL) (29 children)
  [000-019] FLAT_SEGMENT (scale=MESO) (1 children)
    [001-001] LOCAL_PEAK (scale=MICRO)
  [003-018] NORMAL_VOLATILITY (scale=MESO) (9 children)
  

In [ ]:
"""
USAGE GUIDE: Hierarchical Time Series Event Labeling System
============================================================

Quick Start Guide and Common Use Cases
"""

import torch
# from hierarchical_event_labeling import (
#     HierarchicalEventDataset,
#     TextCorpusGenerator,
#     EventScale,
#     VOCAB
# )


# ============================================================================
# QUICK START
# ============================================================================

def quick_start_example():
    """Minimal example to get started"""
    
    # 1. Prepare your data as [B, L] tensor
    B, L = 100, 336  # 100 sequences, each 336 timesteps
    x = torch.randn(B, L)  # Replace with your real data
    
    # 2. Create dataset (this does all the processing)
    dataset = HierarchicalEventDataset(x, verbose=True)
    
    # 3. Access annotations
    annotation = dataset[0]  # Get first sequence annotation
    
    # 4. Generate training text
    text = annotation.to_text(format='depth_marked')
    print(text)
    
    return dataset


# ============================================================================
# LOADING REAL DATA
# ============================================================================

def load_from_numpy():
    """Load from numpy arrays"""
    import numpy as np
    
    # Load your numpy data
    data = np.load('your_data.npy')  # Shape: [num_samples, sequence_length]
    
    # Convert to tensor
    x = torch.from_numpy(data).float()
    
    # Create dataset
    dataset = HierarchicalEventDataset(x)
    
    return dataset


def load_from_csv():
    """Load from CSV files"""
    import pandas as pd
    
    # Load CSV
    df = pd.read_csv('your_data.csv')
    
    # Assuming each row is a sequence
    data = df.values  # Shape: [num_sequences, sequence_length]
    x = torch.from_numpy(data).float()
    
    dataset = HierarchicalEventDataset(x)
    
    return dataset


def load_eeg_example():
    """Example for EEG data"""
    
    # Assuming you have EEG data: [num_trials, num_channels, time_points]
    eeg_data = torch.randn(100, 64, 1000)  # Replace with real data
    
    # Process each channel separately
    datasets = []
    for channel in range(64):
        channel_data = eeg_data[:, channel, :]  # [num_trials, time_points]
        dataset = HierarchicalEventDataset(channel_data, verbose=False)
        datasets.append(dataset)
    
    return datasets


# ============================================================================
# EXPLORING ANNOTATIONS
# ============================================================================

def explore_annotation(dataset):
    """Explore annotation structure"""
    
    ann = dataset[0]
    
    print("="*80)
    print("ANNOTATION EXPLORATION")
    print("="*80)
    
    # 1. View hierarchical structure
    print("\n1. HIERARCHICAL TREE:")
    ann.print_hierarchy(max_depth=3)
    
    # 2. Get events at specific scale
    print("\n2. EVENTS BY SCALE:")
    for scale in EventScale:
        events = ann.get_events_at_scale(scale)
        print(f"{scale.name}: {len(events)} events")
    
    # 3. Get events in time range
    print("\n3. EVENTS IN TIME RANGE [100-200]:")
    events = ann.get_events_in_range(100, 200)
    for e in events:
        print(f"  [{e.start:03d}-{e.end:03d}] {e.label_name} ({e.scale.name})")
    
    # 4. Access event metadata
    print("\n4. EVENT METADATA:")
    for event in ann.all_events[:5]:
        print(f"{event.label_name}: {event.metadata}")
    
    # 5. Step-wise labels
    print(f"\n5. STEP-WISE LABELS (first 20):")
    print(f"IDs: {ann.step_labels[:20].tolist()}")
    print(f"Names: {[VOCAB.id_to_label(int(l)) for l in ann.step_labels[:20]]}")


# ============================================================================
# TEXT GENERATION FOR TRAINING
# ============================================================================

def generate_training_corpus(dataset, output_file='training_corpus.txt'):
    """Generate complete training corpus"""
    
    # Generate text for all sequences
    text_gen = TextCorpusGenerator()
    
    # Try different formats
    formats = ['depth_marked', 'flat', 'narrative']
    
    for fmt in formats:
        print(f"\nGenerating {fmt} format...")
        corpus = text_gen.generate_corpus(dataset, format=fmt)
        
        # Save to file
        filename = f'{fmt}_{output_file}'
        with open(filename, 'w') as f:
            for i, text in enumerate(corpus):
                f.write(f"<sequence_{i}>\n{text}\n</sequence_{i}>\n\n")
        
        # Print statistics
        stats = text_gen.estimate_tokens(corpus)
        print(f"Saved to {filename}")
        print(f"  Documents: {stats['num_documents']}")
        print(f"  Total tokens: {stats['total_tokens']:,}")
        print(f"  Avg tokens/doc: {stats['avg_tokens_per_doc']:.1f}")


def create_autoregressive_pairs(dataset):
    """Create input-output pairs for autoregressive LM training"""
    
    pairs = []
    
    for ann in dataset.annotations:
        # Get hierarchical text
        full_text = ann.to_text(format='depth_marked')
        tokens = full_text.split()
        
        # Create prefix-completion pairs
        for i in range(1, len(tokens)):
            input_text = " ".join(tokens[:i])
            target_token = tokens[i]
            pairs.append((input_text, target_token))
    
    return pairs


# ============================================================================
# FILTERING AND ANALYSIS
# ============================================================================

def filter_by_event_type(dataset, event_type='trend'):
    """Filter sequences by event type"""
    
    filtered = []
    for i, ann in enumerate(dataset.annotations):
        # Check if this annotation contains the event type
        has_event = any(e.event_type == event_type for e in ann.all_events)
        if has_event:
            filtered.append(i)
    
    print(f"Found {len(filtered)} sequences with {event_type} events")
    return filtered


def analyze_event_statistics(dataset):
    """Compute detailed event statistics"""
    
    stats = {
        'total_sequences': len(dataset),
        'events_by_type': {},
        'events_by_scale': {},
        'events_by_label': {},
    }
    
    # Count events
    for ann in dataset.annotations:
        for event in ann.all_events:
            # By type
            stats['events_by_type'][event.event_type] = \
                stats['events_by_type'].get(event.event_type, 0) + 1
            
            # By scale
            stats['events_by_scale'][event.scale.name] = \
                stats['events_by_scale'].get(event.scale.name, 0) + 1
            
            # By label
            stats['events_by_label'][event.label_name] = \
                stats['events_by_label'].get(event.label_name, 0) + 1
    
    # Print statistics
    print("\n" + "="*80)
    print("DETAILED EVENT STATISTICS")
    print("="*80)
    
    print("\nBy Event Type:")
    for event_type, count in sorted(stats['events_by_type'].items()):
        avg = count / stats['total_sequences']
        print(f"  {event_type:.<20} {count:>6} ({avg:.2f} per sequence)")
    
    print("\nBy Scale:")
    for scale, count in sorted(stats['events_by_scale'].items()):
        avg = count / stats['total_sequences']
        print(f"  {scale:.<20} {count:>6} ({avg:.2f} per sequence)")
    
    print("\nTop 10 Labels:")
    top_labels = sorted(stats['events_by_label'].items(), 
                       key=lambda x: x[1], reverse=True)[:10]
    for label, count in top_labels:
        avg = count / stats['total_sequences']
        print(f"  {label:.<30} {count:>6} ({avg:.2f} per sequence)")
    
    return stats


# ============================================================================
# CUSTOM TEXT FORMATS
# ============================================================================

def create_custom_format(ann):
    """Create your own text format"""
    
    parts = []
    
    # Add sequence metadata
    parts.append(f"SEQ_LEN:{len(ann.sequence)}")
    
    # Add global regime
    global_events = ann.get_events_at_scale(EventScale.GLOBAL)
    if global_events:
        parts.append(f"REGIME:{global_events[0].label_name}")
    
    # Add macro trends
    macro_events = ann.get_events_at_scale(EventScale.MACRO)
    parts.append(f"TRENDS:{len(macro_events)}")
    for event in macro_events:
        parts.append(f"T[{event.start}-{event.end}]:{event.label_name}")
    
    # Add peaks
    peaks = [e for e in ann.all_events if e.event_type == 'peak']
    parts.append(f"PEAKS:{len(peaks)}")
    for peak in peaks:
        parts.append(f"P[{peak.start}]:{peak.label_name}")
    
    return " | ".join(parts)


# ============================================================================
# BATCH PROCESSING
# ============================================================================

def process_large_dataset_in_batches(data_generator, batch_size=1000):
    """Process very large datasets in batches"""
    
    all_annotations = []
    
    for batch_idx, batch_data in enumerate(data_generator):
        print(f"\nProcessing batch {batch_idx}...")
        
        # Create dataset for this batch
        dataset = HierarchicalEventDataset(batch_data, verbose=False)
        
        # Collect annotations
        all_annotations.extend(dataset.annotations)
        
        # Optionally save intermediate results
        torch.save(dataset.annotations, f'annotations_batch_{batch_idx}.pt')
    
    print(f"\nTotal annotations: {len(all_annotations)}")
    return all_annotations


# ============================================================================
# INTEGRATION WITH TRAINING PIPELINES
# ============================================================================

def create_pytorch_dataloader(dataset, batch_size=32):
    """Create PyTorch DataLoader for training"""
    from torch.utils.data import DataLoader
    
    def collate_fn(batch):
        """Custom collate function for hierarchical annotations"""
        sequences = torch.stack([ann.sequence for ann in batch])
        step_labels = torch.stack([ann.step_labels for ann in batch])
        
        # Generate text representations
        texts = [ann.to_text(format='depth_marked') for ann in batch]
        
        return {
            'sequences': sequences,
            'step_labels': step_labels,
            'texts': texts,
            'annotations': batch  # Keep full annotations if needed
        }
    
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn
    )
    
    return dataloader


def prepare_for_huggingface(dataset, tokenizer):
    """Prepare data for HuggingFace transformers"""
    
    # Generate text corpus
    text_gen = TextCorpusGenerator()
    texts = text_gen.generate_corpus(dataset, format='depth_marked')
    
    # Tokenize
    tokenized = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )
    
    return tokenized


# ============================================================================
# EXAMPLE WORKFLOWS
# ============================================================================

def complete_workflow_example():
    """Complete end-to-end workflow"""
    
    print("\n" + "="*80)
    print("COMPLETE WORKFLOW EXAMPLE")
    print("="*80)
    
    # 1. Generate/Load data
    print("\n[1/6] Loading data...")
    B, L = 100, 336
    x = torch.randn(B, L)  # Replace with real data
    
    # 2. Create dataset
    print("\n[2/6] Creating hierarchical dataset...")
    dataset = HierarchicalEventDataset(x, verbose=False)
    print(f"  ✓ Processed {len(dataset)} sequences")
    
    # 3. Explore one example
    print("\n[3/6] Exploring example annotation...")
    explore_annotation(dataset)
    
    # 4. Analyze statistics
    print("\n[4/6] Computing statistics...")
    stats = analyze_event_statistics(dataset)
    
    # 5. Generate training corpus
    print("\n[5/6] Generating training corpus...")
    generate_training_corpus(dataset, 'output_corpus.txt')
    
    # 6. Create DataLoader
    print("\n[6/6] Creating DataLoader...")
    dataloader = create_pytorch_dataloader(dataset, batch_size=16)
    print(f"  ✓ DataLoader ready with {len(dataloader)} batches")
    
    print("\n" + "="*80)
    print("✓ WORKFLOW COMPLETE")
    print("="*80)


# ============================================================================
# MAIN
# ============================================================================

if __name__ == "__main__":
    # Run the complete workflow
    complete_workflow_example()

ModuleNotFoundError: No module named 'hierarchical_event_labeling'